# Интеграция с тинькофф

## Импорты

In [4]:
import tinvest as ti
import ta
import asyncio
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
from pytz import timezone
# import pandas_datareader as pdr
import datetime, time
pd.set_option('display.max_columns', 5)
# import mpl_finance as mpf
# import mplfinance as mpf
# from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
# from sklearn.model_selection import cross_val_score, train_test_split, TimeSeriesSplit
# from sklearn.metrics import mean_squared_error, r2_score, make_scorer, mean_absolute_error, f1_score, accuracy_score

In [11]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
# from sklearn.dummy import DummyClassifier
# from ipynb.strategy_pipeline_ls import deal_now
# %run ST_NEW.ipynb

In [12]:
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean, median
import itertools
from google.protobuf.timestamp_pb2 import Timestamp
from tqdm import tqdm
import itertools
import time
import random
import warnings

In [79]:
#  Настройка ноутбука 
from IPython.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [4]:
#!/usr/bin/env python
import psutil
# gives a single float value
psutil.cpu_percent()
# gives an object with many fields
psutil.virtual_memory()
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

{'total': 8121290752,
 'available': 1283100672,
 'percent': 84.2,
 'used': 6838190080,
 'free': 1283100672}

## Функции расчета метрик

In [ ]:
def IND(data, window):
    """ Функция добавления ATR """
    try:
        data['ATR'] = ta.volatility.AverageTrueRange(high=data['High'], low=data['Low'], 
                                                 close=data['Close'], 
                                                 window=window, fillna=False).average_true_range()
    except:
        data['ATR'] = 0
    return data

In [ ]:
def N_add(data, add_corr_N):
    """ Функция добавления N и изменения типа Date. N рассчитывается как среднее за 2 предыдущих дня """
    data.loc[20, 'N'] = data.loc[0:20, 'ATR'].mean() # первое значение N равно среднему
    for i in range(21, len(data), 1):
        # data.loc[i-5:i, 'N'].mean() - сглаживаю за неделю
        data.loc[i, 'N'] = round((19 * data.loc[i-1, 'N']  + data.loc[i, 'ATR'] )/20, 4)
#         data.loc[i, 'N_median'] = round((19 * data.loc[i-5:i-1, 'N'].median()  + data.loc[i, 'ATR'] )/20, 4)  # Все последующие значения рассчитываются по предыдущим N
        #     data['Date'] = pd.to_datetime(data['Date'], format="%Y/%m/%d, %H:%M:%S") # При записи в файл строку включать!
    return data

def arr_N_add(data):
    """ ARR-Функция добавления N и изменения типа Date. N рассчитывается как среднее за 2 предыдущих дня"""
    data.loc[20, 'N'] = data.loc[0:20, 'ATR'].mean() # первое значение N равно среднему
    arr_data = data.to_numpy()
    new_arr = np.empty((0, 8), int) # Пустая 9-ми значная матрица
    for i in range(21, len(arr_data), 1):
        result = round((19 * median(arr_data[i-7:i-1, 7])  + arr_data[i, 7])/20, 4)
        new_arr = np.vstack((new_arr, np.hstack((arr_data[i, 0:7], result))))
    new_data = pd.DataFrame(data=new_arr, columns = ['Open', 'Close', 'High', 'Low', 'Date','Volume', 'ATR', 'N'])
    return new_data

In [ ]:
def ind_min_max_difference(data, mmd_period):
    """ Функция рассчитывает разницу между максимальным и минимальным значением за определенный период"""
    for i in range(mmd_period, len(data)+1, 1):
        maximum = data.loc[i-mmd_period : i-1, 'High'].max()
        minimum = data.loc[i-mmd_period : i-1, 'Low'].min()
        data.loc[i, 'MMD'] = (((maximum - minimum) / (maximum + minimum)) ** 2) ** 0.5

    return data

def arr_ind_min_max_difference(data, mmd_period):
    arr_data = data.to_numpy()
    new_arr = np.empty((0, 9), int) # Пустая 9-ми значная матрица
    for i in range(mmd_period, len(arr_data), 1):
        maximum = arr_data[i-mmd_period : i-1, 2].max()
        minimum = arr_data[i-mmd_period : i-1, 3].min()
        result = (((maximum - minimum) / (maximum + minimum)) ** 2) ** 0.5
        new_arr = np.vstack((new_arr, np.hstack((arr_data[i], result))))
    new_data = pd.DataFrame(data=new_arr, columns = ['Open', 'Close', 'High', 'Low', 'Date','Volume', 'ATR', 'N', 'MMD'])
    return new_data
        

In [ ]:
def normalized_volume_osc(data):
    """Функция рассчитывает значение нормализованного осцилятора объема. 
        10-дневная сумма [(закрытие - открытие) / (максимум - минимум) * объем] / 10-дневная сумма объема"""
    for i in range(10, len(data), 1):
        days_list = []
        for j in range(10, 0, -1):
            close = data.loc[j, 'Close']
            open_q = data.loc[j, 'Open']
            high = data.loc[j, 'High']
            low = data.loc[j, 'Low']
            volume = data.loc[j, 'Volume']
            summ = (close - open_q) / (high - low) * volume
            days_list.append(summ)
        data.loc[i, 'NVO'] = sum(days_list)
    return data  

def arr_normalized_volume_osc(stocks_dict_full_csv):
    """Функция рассчитывает значение нормализованного осцилятора объема. 
        10-дневная сумма [(закрытие - открытие) / (максимум - минимум) * объем] / 10-дневная сумма объема"""
    arr_stocks_dict_full_csv = arr_quotes_from_csv(stocks_dict_full_csv)
    columns = ['Open', 'Close', 'High', 'Low', 'Date','Volume', 'ATR', 'N', 'NVO']
    for key in tqdm(arr_stocks_dict_full_csv.keys()): # Итерация по каждому инструменты
        arr_data = arr_stocks_dict_full_csv[key]
        arr_2 = np.empty((0, 9), int) # Пустая 8-ми значная матрица
        for i in range(15, len(arr_data), 1): # Итерация по каждой строке
            
            days_list = []
            
            for j in range(10, 0, -1): 
                close = arr_data[j, 1]
                open_q = arr_data[j, 0]
                high = arr_data[j, 2]
                low = arr_data[j, 3]
                volume = arr_data[j, 5]            
                summ = (close - open_q) / (high - low) * volume if high - low != 0 else 0.01
                days_list.append(summ)
                
            result = sum(days_list) / arr_data[i-10 : i+1, 5].sum() if arr_data[i-10 : i+1, 5].sum() != 0 else 0.01 
#             print(key,  result)
            arr_2 = np.vstack((arr_2, np.hstack((arr_data[i], result)))) #в будующий дф котировок добавляем строку 
            
        data = pd.DataFrame(data=arr_2, columns = columns )
        arr_stocks_dict_full_csv[key] = data
    return arr_stocks_dict_full_csv
        
        
        

def arr_quotes_from_csv(stocks_dict_full_csv):
    """Функция преобразует датафрейм тикеров в словаре в numpy-массивы"""
    arr_stocks_dict_full_csv = {}
    for i in stocks_dict_full_csv:
        arr_stocks_dict_full_csv[i] = stocks_dict_full_csv[i].to_numpy()
    return arr_stocks_dict_full_csv

In [ ]:
def change_N(data):
    # Меняю значение N
    for quotes in tqdm(stocks_dict_full_csv.values()):
        quotes = quotes.reset_index(drop=True)
        for i in range(21, len(quotes), 1):
            quotes.loc[i, 'N'] = round((19 * quotes.loc[i-2:i-1, 'N'].mean() + quotes.loc[i, 'ATR'] )/20, 4)
    return quotes

# Данные с Тинькова

In [40]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

token = 'Сюда ничего вписывать не надо'
token_2 = 'Сюда вписать свой токен'
client = ti.SyncClient(token)

## Подключение V.2

In [6]:
import datetime #import datetime,timezone
from tinkoff.invest  import Client, RequestError, CandleInterval, HistoricCandle,Share
import pytz



In [7]:
def cast_money(v):
    return v.units + v.nano / 1e9

In [8]:
def create_df(candles : [HistoricCandle]):
    df = pd.DataFrame([{
        'time' : c.time, 
        'open' : cast_money(c.open),
        'close' : cast_money(c.close),
        'high' : cast_money(c.high),
        'low' : cast_money(c.low),
        'volume' : c.volume} for c in candles])
    return df

In [26]:
def get_candles(figi, date_from, date_to):
    with Client(token_2) as client:
        #date_tinvest_2 = #datetime.datetime(2021, 2, 19)
        date_to = datetime.datetime.now()

        r = client.market_data.get_candles(figi = figi, 
                                           from_= date_from,
                                           to=date_to,
                                           interval=CandleInterval.CANDLE_INTERVAL_DAY)
        df = create_df(r.candles)

        return df


# date_tinvest = datetime.datetime(2021, 2, 25)
# df_candles = get_candles('BBG000HLJ7M4', date_tinvest)
# display(df_candles)

In [30]:
def get_stocks():
    """ Функция получает уникальные инструменты (большой ДФ с полной информацией по акции)"""
    with Client(token_2) as client:

        sh = client.instruments.shares(instrument_status=1)
        df_sh = pd.DataFrame(sh.instruments)
        return df_sh

# stocks = get_candles('BBG000HLJ7M4', date_from = datetime.datetime(2022, 2, 1), date_to = datetime.datetime(2019, 8, 30))
# print(df_sh.loc[:, 'sector'].unique()) ### Уникальные отрасли

In [ ]:
def get_futures():
    with Client(token_2) as client:

        ft = client.instruments.futures(instrument_status=1)
        df_ft = pd.DataFrame(ft.instruments)
        return df_ft

In [31]:
# stocks

,time,open,close,high,low,volume
0,2022-01-31 04:00:00+00:00,66.35,69.03,69.25,66.33,116940
1,2022-02-01 04:00:00+00:00,69.68,67.38,69.97,67.17,70235
2,2022-02-02 04:00:00+00:00,67.22,68.59,68.74,67.22,99651
3,2022-02-03 04:00:00+00:00,67.57,67.25,68.55,67.03,61145
4,2022-02-04 04:00:00+00:00,67.06,66.50,67.43,65.79,43149
...,...,...,...,...,...,...
146,2022-09-06 04:00:00+00:00,47.59,46.62,47.93,46.29,115728
147,2022-09-07 04:00:00+00:00,46.02,46.40,46.89,45.22,111508
148,2022-09-08 04:00:00+00:00,46.14,46.79,46.93,45.54,129384
149,2022-09-09 04:00:00+00:00,47.70,47.26,48.07,47.22,99398


## Запись котировок в CSV


In [3]:
def get_stocks_to_CSV(df_sh):
    """ Функция записывает данные, полученные из get_stocks в .CSV"""
    warnings.filterwarnings("ignore")
    df_sh.to_csv(r'C:\Users\ogurskiy\Desktop\DF\df_stocks_info.csv', sep=';', index=False, encoding='cp866')

In [ ]:
def quotes_to_csv(
                  stocks_dict, # Словарь с котировками акций
                  file_name):
    """Функция записи в таблицу .CSV. Вся обработка для представления данных для тестов происходит В ИМПОРТЕ и возвращает 
    котировки, котировки которых не удалось записать"""
    warnings.filterwarnings("ignore")
    tickers = list(stocks_dict.keys()) # Список тикеров котировок акций 
    data_ticker_list = pd.DataFrame()
    error_ticker = []
    
#     #Добавляем ДФ(список списков) в один большой список
    for ticker in tqdm(tickers):
        stock_quotes = stocks_dict[ticker].copy()
        try:
#             stock_quotes["Date"] = stock_quotes["Date"].apply(lambda x: x.strftime('%Y-%m-%d'))#    datetime.date()# Изменяем формат даты
            stock_quotes["Date"] = pd.to_datetime(stock_quotes['Date']).dt.strftime('%Y-%m-%d')
            stock_quotes = pd.Series(stock_quotes.values.tolist()) # ДФ-список котировок акций
            data_ticker_list.loc[:, ticker] = stock_quotes # Добавляем каждую серию в бальшой ДФ (ДФ котировок всех акций)
        except KeyError:
            error_ticker.append(ticker)
            continue
        
    data_ticker_list.to_csv(r'C:\Users\ogurskiy\Desktop\DF\{}.csv'.format(file_name), sep=';', index=False)
#     display(data_ticker_list)
    return error_ticker


## Импорт котировок из CSV

In [ ]:
def str_to_date(data):
    """ Переводит строку даты во время"""
    for i in range(len(data)):
        data_sticker.loc[i,'Date'] = data_sticker.loc[i,'Date'].datetime.datetime.strptime(data_sticker.loc[i,'Date'], '%Y-%m-%d').date()
    return data   

In [ ]:
def quotes_from_csv(file_name):
    """ Функция импортирует нужный файл с котировками и делает следующюю обработку:
        - Разделяет ячейку csv по запятым
        - именует столбцы для будущего ДФ
        - Выкидывает nan из ДФ
        - убирает символы ' ', '[', ']'. '''
        - Приводит дату к виду '%Y-%m-%d'
        - делает индексы датой
        - переводит в соответсвующий тип столбцы (int, float)"""
#     columns_types = {0:float, 1:float,2:float, 3:float,6:float, 7:float, 5:float}
    data = pd.read_csv(r'C:\Users\ogurskiy\Desktop\DF\{}.csv'.format(file_name), sep=';')
    tickers = list(data.columns)
    stocks_dict_full_all = {}
    for ticker in tqdm(tickers):
        data_ticker = data[ticker].str.split(',', expand=True) # Рабочий датафрейм по тикеру
        data_ticker.columns = ['Open', 'Close', 'High', 'Low', 'Date', 'Volume', 'ATR', 'N'] #Название столбцов ДФ
        
        data_ticker = data_ticker.replace(to_replace =[' ', "'", "\[", "\]"], value = '', regex = True) # Убираем лишние символы
        data_ticker = data_ticker.replace(to_replace =['nan'], value = np.nan, regex = True) #'nan' заменяем на np.nan, чтобы потои удалить
        
        data_ticker= data_ticker.dropna() # Выкидывает nan из лишних строк в ДФ () 
#         print(data_ticker['Date'].head(1))
        data_ticker['Date'] = data_ticker['Date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y-%m-%d').date()) # Преобразую во временной тип
        data_ticker.index = data_ticker['Date'] # делаем индексы - датой
        
        
        data_ticker[['Open', 'Close', 'High', 'Low', 'ATR', 'N']] = data_ticker[['Open', 'Close', 'High', 'Low', 'ATR', 'N']].astype('float') 
        data_ticker['Volume'] = data_ticker['Volume'].astype('float')
        stocks_dict_full_all[ticker] = data_ticker
        

    return stocks_dict_full_all

In [ ]:
def arr_quotes_from_csv(stocks_dict_full_csv):
    """Функция преобразует датафрейм тикеров в словаре в numpy-массивы"""
    arr_stocks_dict_full_csv = {}
    for i in stocks_dict_full_csv:
        arr_stocks_dict_full_csv[i] = stocks_dict_full_csv[i].to_numpy()
    return arr_stocks_dict_full_csv
        
        

In [ ]:
def get_stocksinfo_from_CSV():
    """ Функция получает ДФ информации по всем акциям. Но нужно менять кодировку(ms_dos) """
    data = pd.read_csv(r'C:\Users\ogurskiy\Desktop\DF\df_stocks_info.csv', sep=';', encoding='cp866')
    data.columns = [' ', 'Ticker', 'Class_code', 'Isin', 'Lot', 'Currency', 'Klong', 'Kshort', 'Dlong',
                    'Dshort', 'Dlong_min', 'Dshort_min', 'Short_enabled_flag', 'Name', 'Exchange', 'Ipo_date',
                    'Issue_size', 'Country_of_risk', 'Country_of_risk_name', 'Sector', 'Issue_size_plan', 'Nominal', 
                    'Trading_status', 'Otc_flag', 'Buy_available_flag', 'Sell_available_flag', 'Div_yield_flag',
                    'Share_type', 'Min_price_increment', 'Api_trade_available_flag']
    return data
# stocks_info = get_stocksinfo_from_CSV()
# display(stocks_info.columns)

## Получение списка акций в тинькофф online  и подготовка их к записи

In [ ]:
# Получение списка акций в тинькофф
# stocks = client.get_market_stocks()
# stocks = list(list(list(list(list(stock)[0])[1])[0])[1])
def create_data(df_sh):
    
    stocks = df_sh.loc[:, 'ticker'] # Тикеры
    data_stocks = pd.DataFrame(columns=['Ticker', 'Currency', 'FIGI', 'Lot'])
    tickers_list = []
    for i in range(len(stocks)):
        data_stocks.loc[i, 'Ticker'] = df_sh.loc[i, 'ticker']
        data_stocks.loc[i, 'Currency'] = df_sh.loc[i, 'currency']
        data_stocks.loc[i, 'FIGI'] = df_sh.loc[i, 'figi']
        data_stocks.loc[i, 'Lot'] = df_sh.loc[i, 'lot']
        data_stocks.loc[i, 'Sector'] = df_sh.loc[i, 'sector']
    return data_stocks

In [ ]:
# df_sh=get_stocks()
# display(df_sh['country_of_risk_name'].unique())
# data_stocks = create_data(df_sh)

In [ ]:
# Функция приведения данных Тинькофф к виду Яху.Финанс

def normal_data(figi, date_from, date_to):
    
#     date_2 = datetime.datetime.now()
#     candles = client.get_market_candles(figi, date_tinvest, date_2, interval=ti.CandleResolution.day )
#     candles = list(candles.payload.candles)
    
    candles = get_candles(figi, date_from, date_to)
    
    data_candles = pd.DataFrame(columns = ['Open' ,'Close','High', 'Low',  'Date', 'Volume'])
    data_candles[ 'Close'] = candles['close']
    data_candles[ 'High'] = candles['high']
    data_candles[ 'Low'] = candles['low']
    data_candles[ 'Open'] = candles['open']
    data_candles['Volume'] = candles['volume']
    data_candles['Date'] = candles['time']
        
        
    data_candles[['Close', 'High', 'Low', 'Open']] = data_candles[['Close', 'High', 'Low', 'Open']] .astype('float')
    data_candles[['Volume']] = data_candles[[ 'Volume']] .astype('int64')
    data_candles['Date'] = pd.to_datetime(data_candles['Date'], format="%Y/%m/%d, %H:%M:%S",utc=False).dt.tz_localize(None)
    
    return data_candles
    
# print(normal_data('BBG000HLJ7M4', date_tinvest))

In [ ]:
# normal_data('BBG000HLJ7M4', date_tinvest = datetime.datetime(2021, 2, 19))

In [ ]:
def rub_usd(data):
    """ Фукция для разделения на рублевые и долларовые акции"""
    usd = data[data.loc[:,'Currency']=='usd'].reset_index(drop=True)
    rub = data[data.loc[:,'Currency']=='rub'].reset_index(drop=True)
    return usd, rub  

In [ ]:
# Функция прогона по списку акций
def predict_profit_tinvest(data,  deposit):
    for i in range(len(usd_stocks)):
        ticker = usd_stocks.loc[i, 'Ticker']

        figi = usd_stocks.loc[i, 'FIGI']
        data_tinvest = normal_data(data.loc[i, 'FIGI'])
        profit = proverka(open_1(data_tinvest, deposit))
        print('{}: {}'.format(ticker, profit))


In [ ]:
# Функция добавления N и изменения типа Date для ТИНЬКОФФ
def N_add_tinvest(data):
    
    data.loc[20, 'N'] = data.loc[0:20, 'ATR'].mean() # первое значение N равно среднему
    for i in range(21, len(data), 1):
        data.loc[i, 'N'] = round((19*data.loc[i-1, 'N'] + data.loc[i, 'ATR'])/20, 2)  # Все последующие значения рассчитываются по предыдущим N
    
    data['Date'] = pd.to_datetime(data['Date'], format="%Y/%m/%d, %H:%M:%S")
    
    return data

In [ ]:
# Найти ключ по значению
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

In [ ]:
# Наиболее прибыльные компании
def profit_companies(data, bool_p):
    data['Profit'] = data['Profit'].astype('float') 
    profit_companies_data = data.pivot_table( index='Ticker', values='Profit', 
                          
                                             aggfunc=('sum')).sort_values(by=('Profit'), ascending=bool_p).head(5)
    display( profit_companies_data)


In [ ]:
def max_pos(data):
#     list_pos= list(data.loc[:, 'Size/money_unit']).apply(lambda x: x.values())
    data = data.sort_values(by='Id')
    data_2 = pd.DataFrame()
    data_2= data.copy()
    data_2['Size_unit'] = data['Size/money_unit'].apply(lambda x: list(x.keys())[0])
    max_pos = data_2.loc[:, 'Money_unit'].max(), 
    max_pos_id = float(round(list(data_2.loc[data_2['Money_unit'] == max_pos, 'Id'])[0], 1))
#     display( data_2.nlargest(1, columns='Money_unit'))
    display('************************', max_pos)
    
    

In [ ]:
# def get_key(d, value):
#     """ Функция получает значение ключ по значению в словаре"""
#     for k, v in d.items():
#         if v == value:
#             return k

## Старый вызов алгоритма

In [ ]:
# %%time
def run_usd(year):
#     %run ST_NEW.ipynb
    deposit = 10000
    date_from = datetime.datetime(year, 1, 20)
    date_to = datetime.datetime(year, 3, 1)
    usd_stocks, rub_stocks = rub_usd(data_stocks)
    analysis_tinvest_usd_all = pd.DataFrame(columns=['Ticker', 'Dir','U_money', 'U_size', 'Date_open', 'Date_close', 'Pr_op','Units', 'Close',  'S_loss',
                                            'Cr_on', 'S_done','Profit',  'Lot', 'Sector', 'N' ])
    analysis_tinvest_usd = pd.DataFrame(columns=['Ticker', 'Profit', 'N/Open'])
    profit_tinvest_usd = []
#     usd_stocks_test = usd_stocks[usd_stocks.loc[:,'Ticker'] == 'F' ].reset_index(drop=True)

    for i in tqdm(range(0, len(usd_stocks), 40)):
        try:
            ticker = usd_stocks.loc[i, 'Ticker']
            lot = usd_stocks.loc[i, 'Lot']
            sector = usd_stocks.loc[i, 'Sector']
            figi_usd = usd_stocks.loc[i,'FIGI']


            data_tinvest_usd_candles = normal_data(figi_usd, date_from, date_to)
#             print(i)
            data_tinvest_usd_candles = N_add(IND(data_tinvest_usd_candles, 10), 0.85)
            display(data_tinvest_usd_candles)
            print(ticker, i)
            data_tinvest_usd = open_1(data_tinvest_usd_candles, deposit, 1, 0.5, 0.5, ticker, lot, sector)
            analysis_tinvest_usd.loc[i, 'Ticker'] = ticker
            analysis_tinvest_usd.loc[i, 'Profit'] = data_tinvest_usd.loc[:, 'Profit'].sum()
            analysis_tinvest_usd.loc[i, 'N/Open'] = data_tinvest_usd.loc[:, 'N'].median() / data_tinvest_usd.loc[:, 'Pr_op'].median()
            analysis_tinvest_usd.loc[i, 'Count_deals'] = data_tinvest_usd.shape[0]
            analysis_tinvest_usd.loc[i, 'Max_loss'] = round(data_tinvest_usd.loc[ :, 'Profit'].min() / deposit, 5)
            
        #         print(+analysis_tinvest_usd)

            analysis_tinvest_usd_all = pd.concat([analysis_tinvest_usd_all, data_tinvest_usd], ignore_index=True)
            analysis_tinvest_usd_all.loc[i, 'Ticker'] = ticker  
            
        #         profit_tnvest = data_tinvest_usd.loc[:, 'Profit'].sum()


        #         profit_tinvest_usd.append(profit_tinvest)

        #         print(profit_tinvest)
        except:
            pass
    return analysis_tinvest_usd_all


# Подготовка для ARR функций

In [ ]:
def pretrain_array(stocks_dict_full_csv, # Словарь акций ticker : котировки
                   usd_stocks # Аааамериканский горки (американские акции)
                   ):
    """Функция переводит все датафреймы в  np.array"""
    arr_stocks_dict_full_csv = arr_quotes_from_csv(stocks_dict_full_csv)
    arr_usd_stocks = usd_stocks.to_numpy()
    return arr_stocks_dict_full_csv, arr_usd_stocks
    
# arr_stocks_dict_full_csv, arr_usd_stocks = pretrain_array(stocks_dict_full_csv, usd_stocks)
# print(arr_usd_stocks)

# Аналитические функции

## Аналитические функции

In [ ]:
def money_turnover(data):
    """Оборот денег. Может использоваться для рассчета комиссии"""
    turnover = list(data.loc[:, 'Size/money_unit'].apply(lambda x: sum(x.values())))
    return int(sum(turnover))

In [ ]:
def analist_info(data ):
    ### Кол-во убыточных сделок подряд
    data['Id'] = data['Id'].astype('float')
    is_negative = [True if num < 0 else False for num in data['Profit']]
    groups = itertools.groupby(is_negative)
    occurrences_of_negatives = [len(list(g)) for k, g in groups if k]
    try:
        result = max(occurrences_of_negatives)
        max_pos = max(data.loc[:,'Size/money_unit'].apply(lambda x: list(x.values())[0]))
#         max_size = list(data.loc[:,'Size/money_unit']).apply(lambda x: get_key(x, max_pos))
#         series_max_pos = data[data.loc[:,'Size/money_unit'] == {max_size:max_pos}]
    except:
        pass
    profit = int(data.loc[:, 'Profit'].sum())
    
    profit_all = int(data['Profit'].sum())
    
    loss_all = int(data[data['Profit']<0]['Profit'].sum())
    profit_short = int(data[data.loc[:, 'Direct']=='Sh']['Profit'].sum())
    count_short = int(data[data.loc[:, 'Direct']=='Sh']['Profit'].count())
    profit_long = int(data[data.loc[:, 'Direct']=='L']['Profit'].sum())
    count_long = int(data[data.loc[:, 'Direct']=='L']['Profit'].count())
###*********************************************************************************
    try:
        max_loss_sum = list(data.loc[:, 'Profit'])
        max_drowdown = 0 
        max_count = 0
        list_max_drowdown = []
        list_max_count = []
        for i in max_loss_sum:
            if i < 0:
                max_drowdown += i
                max_count += 1
                list_max_drowdown.append(max_drowdown)
                list_max_count.append(max_count)
            else:
                max_count = 0
                max_drowdown = 0
        max_drowdown = round(min(list_max_drowdown), 2)

###*********************************************************************************    
        print('Уникальных инструментов:', len(data.loc[:,'Ticker'].unique()))
        print('Прибыль по всем позициям:', profit_all)

        print('Максимальная позиция:', max_pos)
        print('---------------------------------------------')
        print("Убыточных сделок подряд:", result) 
        print('Максимальная просадка:',max_drowdown) 
        print('---------------------------------------------')
        print('Прибыль/кол-во по коротким позициям: {}/{}'.format(profit_short, count_short)) 
        print('Прибыль/кол-во по длинным позициям: {}/{}'.format(profit_long, count_long))
        print('---------------------------------------------')
        print('Суммарно убыточных сделок:',round(data[data['Profit']<0]['Profit'].sum(),0))
        print('Суммарно прибыльных сделок:',round(data[data['Profit'] > 0]['Profit'].sum(),0))
        print('Оборот:', money_turnover(data))
        print('---------------------------------------------')
#     print('Наиболее прибыльные компании:',profit_companies(analysis_tinvest_usd_all, False))
#     print('Наиболее убыточные компании:',profit_companies(analysis_tinvest_usd_all, True))
   ###### КОМИССИИ 
#     comission = list(analysis_tinvest_usd_all.apply(lambda x: x[13]*x[2]*x[4]*len(x[5]) , axis=1 ))
#     # print(comission)
#     print('Сумма комиссий:',sum(comission)*0.0004) 
###*********************************************************************************
#         max_loss_sum = list(data.loc[:, 'Profit'])
#         max_drowdown = 0 
#         max_count = 0
#         list_max_drowdown = []
#         list_max_count = []
#         for i in max_loss_sum:
#             if i < 0:
#                 max_drowdown += i
#                 max_count += 1
#                 list_max_drowdown.append(max_drowdown)
#                 list_max_count.append(max_count)
#             else:
#                 max_count = 0
#                 max_drowdown = 0
#         max_drowdown = min(list_max_drowdown)
#         print('Максимальная просадка:',max_drowdown) 
###*********************************************************************************    
    except:
        pass
#     return (profit_all, max_pos, int(max_drowdown), np.asarray([profit_short, count_short]), 
#             np.asarray([profit_long, count_long]), int(data[data['Profit']>0]['Profit'].sum()),
#             int(data[data['Profit'] < 0]['Profit'].sum()))

In [ ]:
def def_max_drowdown(data):
    """ Функция рассчитывает величину максимальной просадки"""
    max_loss_sum = list(data.loc[:, 'Profit'])
    max_drowdown = 0 
    max_count = 0
    list_max_drowdown = []
    list_max_count = []
    for i in max_loss_sum:
        if i < 0:
            max_drowdown += i
            max_count += 1
            list_max_drowdown.append(max_drowdown)
            list_max_count.append(max_count)
        else:
            max_count = 0
            max_drowdown = 0
    max_drowdown = round(min(list_max_drowdown), 2)
    return max_drowdown

In [ ]:
#Статистическая таблица по сделкам за год
# analysis_tinvest_usd_all = analysis_tinvest_usd_all[analysis_tinvest_usd_all['Unit_size'] == 0 ]
def create_analytical(analysis_tinvest_usd_all):
#     analysis_tinvest_usd_all = analysis_tinvest_usd_all.loc[~analysis_tinvest_usd_all['Sector'].isin(['utilities', 
#                                                                                                       'telecom'])]
    analytical_table_columns = ['Pr_count', 'Loss_count', 'Max_pr', 'Max_loss', 'Max_loss_count', 'Max_drawdown', 'Short_pr',
                                'Short_count', 'Long_pr', 'Long_count', 'Open_pos']
    analytical_table = pd.DataFrame(columns=analytical_table_columns)
    count = 0
    numOfRows = analysis_tinvest_usd_all.apply(lambda x: 1 if x[9] > 0 else 0 , axis=1 )
    numOfRows_str = list(numOfRows.value_counts())
    try:
        analytical_table.loc[0, 'Pr_count'] = str(numOfRows_str[1])
        analytical_table.loc[0, 'Loss_count'] = str(numOfRows_str[0])
    except:
        pass
    analytical_table.loc[0, 'Max_pr'] = analysis_tinvest_usd_all.loc[:,'Profit'].max()
    analytical_table.loc[0, 'Max_loss'] = analysis_tinvest_usd_all.loc[:,'Profit'].min()
    
    
    
    max_loss_sum = list(analysis_tinvest_usd_all.loc[:, 'Profit'])
    max_drowdown = 0 
    max_count = 0
    list_max_drowdown = []
    list_max_count = []
    for i in max_loss_sum:
        if i < 0:
            max_drowdown += i
            max_count += 1
            list_max_drowdown.append(max_drowdown)
            list_max_count.append(max_count)
        else:
            max_count = 0
            max_drowdown = 0
    try:
        analytical_table.loc[0, 'Max_drawdown'] = min(list_max_drowdown)
        display(analytical_table)
    except:
        pass    
    analist_info(analysis_tinvest_usd_all)
    top_n = analysis_tinvest_usd_all.sort_values('Profit', ascending=False).head(15) #ТОП 5 сделок
    tail_n = analysis_tinvest_usd_all.sort_values('Profit', ascending=True).head(15)


In [21]:
a = np.array([1,2,3,4,5])
b = np.array([12,13])
if True in np.in1d(a,b):
    print([np.in1d(a,b)])

## Запись результатов запуска в таблицу

In [ ]:
# ### Создаем таблицу с известными колонками
# df_results = pd.DataFrame(columns = ['Date','Depo', 'Profit', 'Unq_tickers', 'Max_pos', 'Max_pr_pos','Max_drawdown', 'Sum_loss', 'Sum_profit', 
#                'Corr_s_l', 'Co_dir', 'Co_sector', 'Co_all', 'Co_tickr', 'D_m_long', 'M_m_per', 'D_n_a', 'Y_f_t', 
#                'Mmd_per', 'Brea_per', 'S_L_co', '2N_co', 'Min_of_10_co', 'S_L_sum', '2N_sum', 'Min_of_10_sum', 
#                'Mon_turnover'])
# df_results.to_csv(r'C:\Users\ogurskiy\Desktop\DF\results.csv', sep=';', index=False)

In [ ]:
def results_to_csv(data, parametrs):
    """Функция импортирует из csv данные которые там есть, делают еще одну запись в таблицу и перезаписывает файл"""
    df_result = pd.read_csv(r'C:\Users\ogurskiy\Desktop\DF\results.csv', sep=';')
    
    columns = ['Date','Depo', 'Profit', 'Unq_tickers', 'Max_pos', 'Max_pr_pos','Max_drawdown', 'Sum_loss', 'Sum_profit', 
               'Corr_s_l', 'Co_dir', 'Co_sector', 'Co_all', 'Co_tickr', 'D_m_long', 'M_m_per', 'D_n_a', 'Y_f_t', 
               'Mmd_per', 'Brea_per', 'S_L_co', '2N_co', 'Min_of_10_co', 'S_L_sum', '2N_sum', 'Min_of_10_sum', 
               'Mon_turnover']
    df = pd.Series(index=columns)
    dict_values = data_all_units_metrics(data, parametrs)
#     display(df.columns)
    df['Date'] = datetime.datetime.now().date()
    df['Depo'] = dict_values['Depo']# Депозит
    df['Profit'] = dict_values['Profit'] # Прибыль 
    df['Unq_tickers'] = dict_values['Unq_tickers'] # Уникальных инструментов
    df['Max_pos'] = dict_values['Max_pos']# Максимальная позиция 
    df['Max_pr_pos'] = dict_values['Max_pr_pos']# Максимальная прибыль по 1 позиции
    df['Max_drawdown'] = dict_values['Max_drawdown']# Максимальная просадка (несколько отрицательных позиций)
    df['Sum_loss'] = dict_values['Sum_loss'] # Суммарно убыточных сделок
    df['Sum_profit'] = dict_values['Sum_profit'] # Суммарно прибыльных сделок
    df['Corr_s_l'] = dict_values['Corr_s_l'] # При каких отклонениях N будет срабатывать стоп-лосс
    df['Co_dir'] = dict_values['Co_dir'] # Количество сделок в 1 направлении
    df['Co_sector'] = dict_values['Co_sector'] # Количество сделок в 1 секторе
    df['Co_all'] = dict_values['Co_all'] # Общее количество сделок
    df['Co_tickr'] = dict_values['Co_tickr'] # Количество сделок в 1 тикере
    df['D_m_long'] = dict_values['D_m_long'] # Максимальное отклонение N при закрытии 2N
    df['M_m_per'] = dict_values['M_m_per'] # Закрытие на минимуме или максимуме за определенный период(кол-во дней)
    df['D_n_a'] = dict_values['D_n_a'] # На сколько N должна подняться цена, чтобы добавить еще один юнит
    df['Y_f_t'] = dict_values['Y_f_t'] # С какого года по какой год было тестирование
    df['Mmd_per'] = dict_values['Mmd_per'] # Период, за который рассчитывается диапазон (min-max) / (min+max)
    df['Brea_per'] = dict_values['Brea_per'] # Период, за который ищется прорыв
    df['S_L_co'] = dict_values['S_L_co'] # Количество закрытий по стоп лоссу кол-во
    df['2N_co'] = dict_values['2N_co'] # Количество закрытий по критерию 2N кол-во
    df['Min_of_10_co'] = dict_values['Min_of_10_co'] # Количество закрытий по минимум n-дней кол-во
    df['S_L_sum'] = dict_values['S_L_sum'] # Количество закрытий по стоп лоссу сумма
    df['2N_sum'] = dict_values['2N_sum'] # Количество закрытий по критерию 2N сумма
    df['Min_of_10_sum'] =  dict_values['Min_of_10_sum'] # Количество закрытий по минимум n-дней сумма
    df['Mon_turnover'] = dict_values['Mon_turnover'] # Оборот
    
#     display(df)
#     df = df.reset_index(drop=True)
#     df_result = df.reset_index(drop=True)
    df_result = pd.concat([df_result, df.to_frame().T], axis=0)
    display(df_result)
    df_result.to_csv(r'C:\Users\ogurskiy\Desktop\DF\results.csv', sep=';', index=False)
    return df


## Функции метрик результатов

In [ ]:
def data_all_units_metrics(data, parametrs):
    """Функция считает метрики и выдает их значения. Описание метрик см. ниже"""
    close_type_pivot_count = data.pivot_table(columns='Criterion', values='Profit', aggfunc='count')
    close_type_pivot_sum = data.pivot_table(columns='Criterion', values='Profit', aggfunc='sum')

    profit = data.loc[:,'Profit'].sum()
    unq_tickers = len(data.loc[:,'Ticker'].unique())
    max_pos = max(data.loc[:,'Size/money_unit'].apply(lambda x: list(x.values())[0]))
    max_pr_pos = data.loc[:,'Profit'].max()
    max_drawdown = def_max_drowdown(data)
    sum_loss = round(data[data['Profit']<0]['Profit'].sum(),0)
    sum_profit = round(data[data['Profit'] > 0]['Profit'].sum(),0)
    depo = parametrs['Depo']
    corr_s_l = parametrs['Corr_s_l']
    co_dir = parametrs['Co_dir']
    co_sector = parametrs['Co_sector']
    co_all = parametrs['Co_all']
    co_tickr = parametrs['Co_tickr']
    d_m_long = parametrs['D_m_long']
    m_m_per = parametrs['M_m_per']
    d_n_a = parametrs['D_n_a']
    y_f_t = parametrs['Y_f_t']
    mmd_per = parametrs['Mmd_per']
    brea_per = parametrs['Brea_per']
    s_l_co = close_type_pivot_count['Stop loss'][0] if 'Stop loss' in close_type_pivot_count.columns else 0
    N2_co = close_type_pivot_count['2N'][0] if '2N' in close_type_pivot_count.columns else 0
    min_of_10_co = close_type_pivot_count['min of 10'][0] if 'min of 10' in close_type_pivot_count.columns else 0
    s_l_sum = close_type_pivot_sum['Stop loss'][0] if 'Stop loss' in close_type_pivot_sum.columns else 0
    N2_sum = close_type_pivot_sum['2N'][0] if '2N' in close_type_pivot_sum.columns else 0
    min_of_10_sum = close_type_pivot_sum['min of 10'][0] if 'min of 10' in close_type_pivot_sum.columns else 0
    mon_turnover = money_turnover(data)
#     columns = ['Profit', 'Unq_tickers', 'Max_pos', 'Max_pr_pos','Max_drawdown', 'Sum_loss', 
#                'Sum_profit', 
#                'Depo',  'Corr_s_l', 'Co_dir', 'Co_sector', 'Co_all', 'Co_tickr', 'D_m_long', 'M_m_per', 'D_n_a', 'Y_f_t', 
#                'Mmd_per', 'Brea_per', 'S_L_co', '2N_co', 'Min_of_10_co', 'S_L_sum', '2N_sum', 'Min_of_10_sum', 
#                'Mon_turnover']
    return dict([('Profit', profit), ('Unq_tickers', unq_tickers), ('Max_pos', max_pos), 
                 ('Max_pr_pos', max_pr_pos), ('Max_drawdown', max_drawdown),
                 ('Sum_loss', sum_loss), ('Sum_profit', sum_profit), ('Depo', depo), 
                 ('Corr_s_l', corr_s_l), ('Co_dir', co_dir), ('Co_sector', co_sector), ('Co_all', co_all),
                 ('Co_tickr', co_tickr), ('D_m_long', d_m_long), ('M_m_per', m_m_per),
                 ('D_n_a', d_n_a), ('Y_f_t', y_f_t), ('Mmd_per', mmd_per), ('Brea_per', brea_per), ('S_L_co', s_l_co), 
                 ('2N_co', N2_co), ('Min_of_10_co', min_of_10_co), ('S_L_sum', s_l_sum), 
                 ('2N_sum', N2_sum), ('Min_of_10_sum', min_of_10_sum), ('Mon_turnover', mon_turnover)])

# V.2 Алгоритм

## Индикаторы

In [ ]:
def BB(data):
    """ Функция добавляет индикатор, указанный ниже"""
#     data['IND'] = ta.volatility.bollinger_pband( close=data['Close'], window_dev = 2,
#                                                  window=window, fillna=False)
    data['SMA'] = ta.trend.SMAIndicator(close=data['Close'], 
                                                window=40, fillna=False).sma_indicator() #Простая скользящая средняя
#     data['IND'] = ta.volume.MFIIndicator(high=data['High'] , low=data['Low'], close=data['Close'] ,
#                                          volume=data['Volume'] ,window = 20, fillna = False).money_flow_index()
#     print(dir(data['IND'].reset_index(drop=True)[40]))
    return data, data.columns[8]

In [51]:
def BB2(data):
    """ Функция добавляет индикатор, указанный ниже"""
#     data.rename(columns={'MFI': 'MI'}, inplace=True)
#     data['MFI'] = ta.volume.MFIIndicator(high=data['High'] , low=data['Low'], close=data['Close'], 
#                                           volume=data['Volume'] ,window = 20, fillna = False).money_flow_index()
#     try:
    data['MI'] = ta.trend.MassIndex(high=data['High'] , low=data['Low'], window_fast=10,window_slow = 40,
                                                 fillna = False).mass_index()
#     except:
#         print(1)
#     print(dir(data['ADX'].reset_index(drop=True)[40]))
#     print(data.columns)
    return data, data.columns[9]

In [ ]:
def BB3(data):
    """ Функция добавляет индикатор, указанный ниже"""
    data['%BB'] = ta.volatility.bollinger_pband( close=data['Close'], window_dev = 2,
                                                 window=60, fillna=False)
#     print(dir(data['IND'].reset_index(drop=True)[40]))
    return data, data.columns[10]

In [ ]:
def custom_volatility(data):
    """Волатильность за какой то период"""
    data = data.reset_index(drop=True)
    start = data['Date'][0] + timedelta(days=10)
    result = pd.Series(index = range(len(data)))
    
    for i in range(10, len(data)):
        body_list = []
        shadow_list = []
        for j in range(i-10, i):

            close = data.loc[j, 'Close']
            open_n = data.loc[j, 'Open']
            high = data.loc[j, 'High']
            low = data.loc[j, 'Low']

            body_list.append(abs(close-open_n))
            shadow_list.append(abs(abs(high-low) - abs(close-open_n)))
        result[i] = median(shadow_list) / median(body_list) / data.loc[i, 'Close']
    data['CV'] = result
    data.index = data['Date']
    return data, 'CV'

In [74]:
def PV(data):
    """periodic_volatility Кастомный индикатор, который вычисляет насколько акция волатильная в пределах периода. 
    Отношение суммы теней к сумме тел."""
#     data['Date'] = data.index
#     data = data.reset_index(drop=True)
#     start = data['Date'][0] + timedelta(days=10)
    result = pd.Series(index = range(len(data)))
    data_numpy = data.to_numpy()
    for i in range(10, len(data)):
# columns=['Open', 'Close', 'High', 'Low', 'Date', 'Volume', 'ATR', 'N', 'SMA', 'MFI', '%BB']
        shadow_array = sum(abs(data_numpy[i-20:i, 2] - data_numpy[i-20:i, 3]))
        result[i] = shadow_array / data_numpy[i, 1]
#         result[i] = sum(shadow_list) / data.loc[i, 'Close']
    data['PVn'] = list(result)
#     data.index = data['Date']
#     print(data)
    return data, 'PVn'

#     return  result 

In [ ]:
def CSMA(data):
    '''Считает колличество пересечений средней за период до сделки'''

    data['CSMA1'] = 0
    data['CSMA'] = 0
    data_numpy = data.to_numpy()
    for i in range(20, len(data)):
#         print(data_numpy[i])
        data_numpy[i, 11] = 1 if ((data_numpy[i, 0] < data_numpy[i, 8] < data_numpy[i, 1]) or
                                  (data_numpy[i, 0] > data_numpy[i, 8] > data_numpy[i, 1])) else 0
        data_numpy[i, 12] = data_numpy[i-20:i, 11].sum()
    data['CSMA'] = data_numpy[:, 12]
    return data, 'CSMA'

In [77]:
def BB4 (data_pos_test):
    """Индикатор рассчитывается, когда алгоритм открытия сделок уже отработал и сформирована таблица data_pos_pivot"""
    ticker_list = data_pos_test['Ticker'].unique() # B7e03MgL Q-1309354 Kiuq1673626
    
    for i in tqdm(ticker_list): ## Применяем алгоритм для котировок
        data = stocks_dict_full_csv_big[i]

#         stocks_dict_full_csv_big[i], name = CSMA(data)
#         stocks_dict_full_csv_big[i], name = PV(stocks_dict_full_csv_big[i])
        stocks_dict_full_csv_big[i], name = MB(stocks_dict_full_csv_big[i])
#     print(stocks_dict_full_csv_big[i])
    data_pos_test['MBl'] = None
    data_pos_test['MBs'] = None
    data_pos_test['Date'] = data_pos_test.index     
    data_pos_test = data_pos_test.reset_index(drop=True)
    
#     data_pos_test[name] = data_pos_test[name].astype('object')
    for i in tqdm(list(data_pos_test.index)):
        ticker = data_pos_test.loc[i, 'Ticker']
        date = data_pos_test.loc[i, 'Date']
#         print(data_pos_test.at[i, name])
#         data_pos_test.loc[i, name] = stocks_dict_full_csv_big[ticker].loc[date, name]
        data_pos_test.at[i, 'MBl'] = stocks_dict_full_csv_big[ticker].loc[date, 'MBl']
        data_pos_test.at[i, 'MBs'] = stocks_dict_full_csv_big[ticker].loc[date, 'MBs']
        
    data_pos_test.index = list(data_pos_test.loc[:,'Date'])
    data_pos_test = data_pos_test.drop(columns='Date')
#     data_pos_test = data_pos_test.drop(columns='PVn')
#     data_pos_test = data_pos_test.drop(columns='CSMA1')
#     data_pos_test = data_pos_test.drop(columns='MB')
    
    return data_pos_test


In [ ]:
def MB(data):
    """Max breakdown. Считает максимальный прорыв для сделки. НЕ РАБОТАЕТ"""
    try:
        data = data.drop(columns='PVn')
        data = data.drop(columns='CSMA1')
        data = data.drop(columns='MB')
    except:
        pass
#     print(data.columns)
# ['Open', 'Close', 'High', 'Low', 'Date', 'Volume', 'ATR', 'N', 'SMA', 'MFI', '%BB', 'PVn', 'CSMA1', 'CSMA']
    data_numpy = data.to_numpy()
    period_long_list = []
    period_short_list = []
    for i in range(0, len(data)):
#         counter_short = 0
        period_long = 1
        period_short = 1
        while data_numpy[i, 2] > data_numpy[i-period_long, 2]:
            period_long +=1
        while data_numpy[i, 2] < data_numpy[i-period_short, 2]:
            period_short +=1
#             counter_short +=1
#         date = data_numpy[i, 4]
        period_long_list.append(period_long)
        period_short_list.append(period_short)
        
        
    data.loc[:, 'MBl'] = period_long_list
    data.loc[:, 'MBs'] = period_short_list
#         data['CSMA'] = data_numpy[:, 12]
    return data, 'MB'

### Подготовка разных массивов

In [ ]:
def calendar_year(date_from, 
                  date_to):
    """ Функция выводит список дат в границах диапазона """
    date_list = []
    calendar = []
    delta = date_to - date_from         # timedelta
    if delta.days <= 0:
        print ("Ругаемся и выходим")
    for i in range(delta.days + 1):
        date_list.append((date_from + timedelta(i)).date())
    for date in date_list:
        if (date.weekday() in [0,1,2,3,4]): # Проверка, что день - будний и есть открытые сделки по тикеру 
            calendar.append(date)
    return calendar


In [ ]:
def calendar_analyst(date_from, 
                  date_to):
    """ Функция выводит список дат в границах диапазона """
    date_list = []
    calendar = []
    delta = date_to - date_from         # timedelta
    if delta.days <= 0:
        print ("Ругаемся и выходим")
    for i in range(delta.days + 1):
        date_list.append((date_from + timedelta(i)))
    for date in date_list:
        if (date.weekday() in [0,1,2,3,4]): # Проверка, что день - будний и есть открытые сделки по тикеру 
            calendar.append(date)
    return calendar


In [ ]:
def stock_quotes(data_stocks_info,
                 add_corr_N,
                 ticker, 
                 date_from,
                 date_to):
    """ Функция создает DataFrame котировок для акции. ИНДЕКСЫ - ДАТЫ """
#     print(data_stocks_info.loc[data_stocks_info.loc[:,'Ticker'] == ticker])
    figi = list(data_stocks_info.loc[data_stocks_info.loc[:,'Ticker'] == ticker]['FIGI'])[0] #вычисляет FIGI по тикеру
    
    data_stock = normal_data(figi, date_from, date_to)
    data_stock = N_add(IND(data_stock, 10), add_corr_N)
    data_stock.index = data_stock['Date'].apply(lambda x: x.date())
#     data_stock['Date'] = data_stock['Date'].apply(lambda x: x.date())
    return data_stock

In [ ]:
def create_stocks_dict_full(stocks_list,
                            data_stocks_info, 
#                             date_from,
#                             date_to,
                            add_corr_N,
                            year_from):
    """ Функция создает словарь, где ключи это тикеры, а значения - датафреймы котировок
        На вход принимает список интересующих акций, DataFrame всех акций с информацией о них (FIGI, отрасль и тд.)
        и даты начала и окончания периода. ИНДЕКСЫ - ДАТЫ"""
    stocks_dict_full = {}
    for ticker in tqdm(stocks_list):
        stocks_dict_full[ticker] = pd.DataFrame()

            #### исправленный абзац
#             date_from = datetime.datetime(year_from, 1, 2)
#             date_to = datetime.datetime(2021, 12, 31)
#             stocks_dict_full[ticker] = stock_quotes(data_stocks_info, add_corr_N , ticker, date_from, date_to)
            #### исправленный абзац

        for year in range(year_from, 2022):
            date_from = datetime.datetime(year, 1, 2)
            date_to = datetime.datetime(year, 12, 31)
            try:
                one_year_data = stock_quotes(data_stocks_info, add_corr_N , ticker, date_from, date_to) # Датафрейм с котировками за 1 год(ограничение тинькофф)

                stocks_dict_full[ticker] = pd.concat([stocks_dict_full[ticker], one_year_data])
            except: 
                print(ticker)
                continue
        stocks_dict_full[ticker] = stocks_dict_full[ticker].reset_index(drop=True)
        stocks_dict_full[ticker] = N_add(IND(stocks_dict_full[ticker], 10), add_corr_N)
        
        try:
            stocks_dict_full[ticker].index = stocks_dict_full[ticker]['Date'].apply(lambda x: x.date())
            stocks_dict_full[ticker]['Date'] = stocks_dict_full[ticker]['Date'].apply(lambda x: x.date())
        except:
            print('keyError:{}'.format(ticker))
        
    return stocks_dict_full

# stocks_dict_add[ticker] = pd.concat([stocks_dict_add[ticker], stocks_dict_full[ticker][stocks_dict_full[ticker].index == date]]) #Добавили строку в словарь котировок


In [ ]:
def create_stocks_dict_add(stocks_list):
    """ Функция создает словарь, где ключи это тикеры, а значения - датафреймы котировок
        Сюда будут добавляться котировки по 1 дню """
    stocks_dict_add = {}
    columns = ['Open', 'Close', 'High', 'Low', 'Date', 'Volume', 'ATR',  'N']
    for stock in stocks_list:
        stocks_dict_add[stock] = pd.DataFrame(columns=columns)
    return stocks_dict_add    

def arr_create_stocks_dict_add(stocks_list):
    """ Функция создает словарь, где ключи это тикеры, а значения - матрицы котировок
        Сюда будут добавляться котировки по 1 дню """
    arr_stocks_dict_add = {}
    for stock in stocks_list:
        arr_stocks_dict_add[stock] = np.empty((0,8), int)
    return arr_stocks_dict_add

### Расчет прорывов

In [ ]:
def breakout_common(period, 
                  stocks_dict_add,
                  date, 
                  ticker, 
                  data):
    """ Функция проверяет наличие прорыва за продолжительность периода (period) LONG """
    try:
        start = date - timedelta(days=period)
        end = date - timedelta(days=1)
        if data.loc[date, 'High'] > data.loc[start : end, 'High'].max()+0.03:
            return 1 # есть прорыв
        elif data.loc[date, 'Low'] < data.loc[start : end, 'Low'].min()-0.03:
#             print(date, data.loc[date, 'Low'], data.loc[start : end, 'Low'].min()-0.03)
            return 2 
        else:            
            return 0
    except:
        print('errorLONG')
        pass
        

############################################################## ARR #####################################################    
def numpy_breakout_common(period, 
                        stocks_dict_add,
                        date, 
                        ticker, 
                        data):
    """ Функция проверяет наличие прорыва за продолжительность периода (period) LONG """
    data_numpy = data.reset_index(drop=True).to_numpy()
#     date = data_numpy[data_numpy[:, 4] == date][:, 2]
#     print(date)
# columns=['Open', 'Close', 'High', 'Low', 'Date', 'Volume', 'ATR', 'N', 'SMA', 'MFI', '%BB']
    
    start = date - timedelta(days=period)
    end = date - timedelta(days=1)
#     print(max(data_numpy[(data_numpy[:, 4] > start) & (data_numpy[:, 4] < end)][:, 2]))
    try:   
        if data_numpy[data_numpy[:, 4] == date][:, 2][0] > max(data_numpy[(data_numpy[:, 4] > start) & (data_numpy[:, 4] < end)][:, 2]) + 0.03:
            return 1
        
        elif data_numpy[data_numpy[:, 4] == date][:, 3][0] < min(data_numpy[(data_numpy[:, 4] > start) & (data_numpy[:, 4] < end)][:, 3]) - 0.03:
            return 2
        
        else:
            return 0
    except:
        print('errorLONGnumpy')
        pass

In [ ]:
############################################################## ARR #####################################################    
def arr_breakout_long(period, 
                      arr_stocks_dict_add,
                      date, 
                      ticker):
    """ Функция проверяет наличие прорыва за продолжительность периода (period) LONG """
    try:
        arr_data = arr_stocks_dict_add[ticker]
        start = date - timedelta(days=period)
        end = date - timedelta(days=1)
        
        if arr_data[arr_data[:, 4] == date][:, 2] > max(arr_data[(arr_data[:, 4] > start) & (arr_data[:, 4] < end)][:, 2]): #Был ли прорыв максимума за прошедший период
            return 1 # есть прорыв
        return 0
    except:
        pass

def breakout_short(period,
                   stocks_dict_add,
                   date, 
                   ticker, 
                   data):
    """ Функция проверяет наличие прорыва за продолжительность периода (period) SHORT """
    try:
        start = date - timedelta(days=period)
        end = date - timedelta(days=1)
        if data.loc[date, 'Low'] < data.loc[start : end, 'Low'].min()-0.03:
#             print ('Date:{}, Start:{}, End:{}'.format(date, start, end))
            return 1 # есть прорыв   
        return 0
    except:
        print('errorSHORT')
        pass

In [17]:
def breakout_common_prev(period,
                         stocks_dict_add,
                         date, 
                         ticker
                         ):
    """ Функция проверяет был ли предыдущий прорыв прибыльным. Функция одинакова для прорыва вниз и вверх.
        Прибыльным считается если цена пробила максимум/минимум и затем произошел прибыльный 10-дневный выход 
        Возвращает 1, если был прибыльный прорыв """
    data = stocks_dict_add[ticker].reset_index(drop=True) # Котировки по акции
    i = max(list(data.index)) if len(list(data.index)) > 0 else 0 # Максимальный индекс
#     print(i)
    for j in range(i-1, 0, -1): # Отсчет в обратной последовательности
        # Условие что предыдущий прорыв был ПРИБЫЛЬНЫЙ
        if ((data.loc[j, 'High'] > data.loc[j-period : j-1, 'High'].max() # Условие прорыв
              and all(data.loc[j-period:, 'Low'] > data.loc[j-period:, 'SMA'])) # Были сделка прибыльной
            or (data.loc[j, 'Low'] < data.loc[j-period : j-1, 'Low'].min()
              and all(data.loc[j-period:, 'High'] < data.loc[j, 'SMA']))
            ):
#             print('Прорыв был:',data.loc[j, 'Date'], '. Сейчас число:',date)
            return 1   
        else:
            return 0 
        # Условие что предыдущий прорыв был УБЫТОЧНЫЙ
#         elif ((data.loc[j, 'High'] < data.loc[j-period : j-1, 'High'].max()) & (data.loc[j+period, 'Close'] < data.loc[j, 'Open'] )
#             or (data.loc[j, 'Low'] > data.loc[j-period : j-1, 'Low'].min()) & (data.loc[j+period, 'Close'] > data.loc[j, 'Open'])):
#             return 0


In [39]:
def test():
    df1 = pd.DataFrame(data=[[111,2,3],[4,5,6]], columns=['c1', 'c2', 'c3'])
    df2 = pd.DataFrame(data=[[11,22,33],[44,55,66]], columns=['C1', 'C2', 'C3'])
    display(df1, df2)
    if any(df1.loc[:,'c1'] < df2.loc[:,'C1']):
        print('')
        print(df1.loc[:,'c1'])
        print(df2.loc[:,'C1'])
# test()

,c1,c2,c3
0,111,2,3
1,4,5,6


,C1,C2,C3
0,11,22,33
1,44,55,66



0    111
1      4
Name: c1, dtype: int64
0    11
1    44
Name: C1, dtype: int64


In [ ]:
                
def arr_breakout_common_prev(period,
                             arr_stocks_dict_add,
                             date, 
                             ticker):
    """ Функция проверяет был ли предыдущий прорыв прибыльным. Функция одинакова для прорыва вниз и вверх.
        Прибыльным считается если цена пробила максимум/минимум и затем произошел прибыльный 10-дневный выход 
        Возвращает 1, если был прибыльный прорыв """
    arr_data = arr_stocks_dict_add[ticker]
    
    i = len(arr_data) if len(arr_data) > 0 else 0


    for j in range(i-period, 0, -1):
        # Условие, чтобы не возникало ошибки, когда период больше чем ко-во дней, прошедших с начала откр. поз
        if j >= period:
            period = period
        elif j < period:
            period = j
        else:
            print("Непредвиденная ошибка в функции arr_breakout_common_prev")
        item_cond_1_1 = arr_data[j,2] # 1 элемент условия №1  data.loc[j, 'High']
        item_cond_1_2 = arr_data[j-period : j-1, 2] # 2 элемент условия №1 data.loc[j-period : j-1, 'High']
        item_cond_2_1 = arr_data[j+period-1, 1] # data.loc[j+10, 'Close']
        item_cond_2_2 = arr_data[j, 0] # data.loc[j, 'Open']
        
        item_cond_3_1 = arr_data[j, 3] # data.loc[j, 'Low']
        item_cond_3_2 = arr_data[j-period : j-1, 3] # data.loc[j-period : j-1, 'Low']
  
   
        # Условие что предыдущий прорыв был ПРИБЫЛЬНЫЙ             
        if (((item_cond_1_1 > max(item_cond_1_2)) & (item_cond_2_1 > arr_data[j, 0])) or
            ((item_cond_3_1 < min(item_cond_3_2)) & (item_cond_2_1 < arr_data[j, 0]))):
            return 1

        # Условие что предыдущий прорыв был УБЫТОЧНЫЙ
        elif (((item_cond_1_1 < max(item_cond_1_2)) & (item_cond_2_1 < arr_data[j, 0])) or
             ((item_cond_3_1 > min(item_cond_3_2)) & (item_cond_2_1 > arr_data[j, 0]))):
            return 0

        else: # 
            return 2
#             print("Непредвиденная ошибка в функции arr_breakout_common_prev 2")   
            

## Открытие позиций и проверка условий

### Открытие новой позиции

In [ ]:
def open_position_long(stocks_dict_add,
                      data_stocks_info,
                      data_actual_units, 
                      columns, 
                      deposit, 
                      date, 
                      ticker, 
                      period, 
                      corr_unit_size, 
                      test_list, 
                      id_pos, 
                      direct, 
                      N,
                      data,
                      mmd_period, 
                      count_ticker_corr): # кол-во открытых тикеров
    """ Функция открывает позицию, размером в 1 юнит """
    data = stocks_dict_add[ticker] # Датафрейм с котировками акции
    sector = data_stocks_info[data_stocks_info.loc[:,'Ticker'] == ticker]['Sector'].unique()[0]
    lot = data_stocks_info[data_stocks_info.loc[:,'Ticker'] == ticker]['Lot'].unique()[0]
    date_open = date
    price_open = data.loc[date, 'Close']#def_price_open(data, date, period, direct)
    # Проверяю условие, чтобы не было юнитов с 0 акций
    size_unit = calk_size_unit(data, data_stocks_info, deposit, date, corr_unit_size, price_open, lot, N, count_ticker_corr)
    try:
        if size_unit > 0:

            money_unit = calk_money_unit(price_open, lot, size_unit)
            date_close =  None
            price_close = None
            criterion = None
            profit = None
            id_pos += 0.01

            atr = data.loc[date,'ATR']
#             mmd = normalized_volume(data, date, mmd_period)
#             mmd = median_volume(data, date, mmd_period)#price_open#open_min_max_difference(data, date, mmd_period) #data.loc[date, 'IND'] 
## Автоматическое переименование колонок
            ind_name = data.columns[8]
            ind2_name = data.columns[9]
            ind3_name = data.columns[10]
            
            ind = data.loc[date, ind_name] 
            ind2 = data.loc[date, ind2_name]
            ind3 = data.loc[date, ind3_name]
            
            
            df = pd.DataFrame(data=[[id_pos, ticker, sector, direct, lot, {size_unit:money_unit}, {date_open:price_open}, 
                                    {date_close:price_close},  criterion, profit, atr, N, ind, ind2, ind3]], columns = columns, index=[date])

            data_actual_units = pd.concat([data_actual_units, df])
            test_list.append(ticker)

            return data_actual_units, id_pos

        else:
#             print('def open_position_long', size_unit, ticker, price_open, N)
            return data_actual_units, id_pos
    except:
        print("EXCEPTION", date, ticker)
 

In [ ]:
def open_min_max_difference(data, date, mmd_period):
    """ Функция рассчитывает показатель периода консолидации за определнный период."""
    start = date - timedelta(days=mmd_period)
    start_candle = date - timedelta(days=2)
    end = date - timedelta(days=1)
    maximum = data.loc[start : end, 'High'].max()
    minimum = data.loc[start : end, 'Low'].min()
    
    min_candle = data.loc[start_candle:end, 'Low'].mean()
    max_candle = data.loc[start_candle:end, 'High'].mean()
    
    ### Чем меньше result, тем уже зона консолидации
    result = abs((maximum - minimum) / ((min_candle + max_candle) / 2)) 
    return result

In [ ]:
def median_volume(data, date, mmd_period):
    """Функция рассчитывает первышение объема над медианным за некоторый период"""
    start = date - timedelta(days=10)
    start_candle = date - timedelta(days=2)
    end = date - timedelta(days=1)
    
    maximum = data.loc[start_candle : end, 'Volume'].max()
    srednee = data.loc[start : end, 'Volume'].median()
     
    return maximum/srednee

In [ ]:
def def_price_open(data,
                   date, 
                   period,
                   direct):  
    """ Функция рассчитывает цену открытия """
    # от start до end ищется максимальная цена за период 
    start = date - timedelta(days=period)
    end = date - timedelta(days=1)
    if (date.weekday() in [0,1,2,3,4]):
        if direct == "Long":
            result = round(data.loc[start : end, 'High'], 2)
            
            result = round(max(data.loc[start : end, 'High'].max(), data.loc[date, 'Open']), 2)
            return result
        elif direct == 'Short':
            result = round(min(data.loc[start : end, 'Low'].min(), data.loc[date, 'Open']), 2)
            return result
    else:
        print('Shit')
        
def arr_def_price_open(arr_data,
                       date, 
                       period,
                       direct):  
    """ Функция рассчитывает цену открытия """
    start = date - timedelta(days=period)
    end = date - timedelta(days=1)
    if (date.weekday() in [0,1,2,3,4]):
        if direct == "Long":
            max_of_period_1 = max(arr_data[(arr_data[:, 4] > start) & (arr_data[:, 4] < end)][:,2]) # Максимум за период
            max_of_period_2 = arr_data[arr_data[:, 4] == date][:,0]
            result = round(int(max(max_of_period_1, max_of_period_2)), 2)
            return result
        elif direct == 'Short':
            min_of_period_1 = min(arr_data[(arr_data[:, 4] > start) & (arr_data[:, 4] < end)][:,3]) # Максимум за период
            min_of_period_2 = arr_data[arr_data[:, 4] == date][:,0]
            result = round(int(min(min_of_period_1, min_of_period_2)), 2)
#             print(int(min(min_of_period_1, min_of_period_2)))
            return result
    else:
        print('Shit')

In [1]:
def calk_size_unit(data, 
                   data_stocks_info, 
                   deposit, 
                   date,
                   corr_unit_size, 
                   price_open,
                   lot,
                   N,
                   count_ticker_corr): #кол-во открытых тикеров
    """ Функция рассчитывает размер юнита в лотах. Условие что позиция не может быть больше депозит/1.25"""
#     size_unit = list(data_ticker.loc[ data_ticker['Id'] == id_pos, 'Size/money_unit'][0].keys())[0]
    
    unit_size = ((0.01*deposit)/ (N/corr_unit_size * price_open))//lot
    unit_size = round(unit_size, 0) * count_ticker_corr*2  if (round(unit_size, 0) * price_open *
                                                             count_ticker_corr *2 < deposit*0.25) else (deposit*0.25 // 
                                                                                                     price_open )
    return unit_size

def arr_calk_size_unit(arr_data, 
                       arr_data_stocks_info, 
                       deposit, 
                       date,
                       corr_unit_size, 
                       price_open,
                       lot):
        """ Функция рассчитывает размер юнита в лотах. Условие что позиция не может быть больше депозит/1.25"""
        if (date.weekday() in [0,1,2,3,4]): # Если день недели - будни
            N = arr_data[arr_data[:, 4] == date][:, 7] # Рассчитывает N
            unit_size = ((0.01*deposit)/ (N/corr_unit_size * price_open))//lot
            unit_size = deposit / 1.25 / price_open if unit_size * price_open > deposit / 1.25 else unit_size 
            return unit_size

In [ ]:
def calk_money_unit(price_open,
                    lot, 
                    size_unit):
    """ Функция рассчитывает размер юнита в деньгах"""
#     print(price_open, lot, size_unit)
    return round(price_open * lot * size_unit, 2)

### Добавление юнита

In [ ]:
def add_unit(usd_stocks, 
             stocks_dict_add, # Словарь, в который добавляются котировки
             columns, 
             deposit,
             date, 
             ticker,
             period,
             corr_unit_size,
             test_list, #???
             N,
             mmd_period,
             count_ticker_corr,
             data_actual_units, # Открытые позиции
             data_all_units,   # датафрейм со всеми позициями (закрытыми)
             data_ticker, # датафрейм тикера из общего датафрейма открытых позиций
             data, # ДФ со значениями котировок (уже добавленных)
             id_pos): # ID позиции
    """Функция добавляет новый юнит на откате к средней к уже открытой позиции"""
#     price_open = list(data_ticker.loc[ data_ticker['Id'] == i, 'Date/price_open'][0].values())[0] 
    price_sma = data.loc[date, 'SMA']
    
    max_index = data_ticker.index.max()
    direct = data_ticker.loc[max_index,'Direct'] # Цена открытия последнего открытого юнита
            
#     direct = data_ticker.loc[ data_ticker['Id'] == i, 'Direct'].index # Дата открытия уже открытой позиции
    start = date - timedelta(days=5)
    end = date - timedelta(days=1)
    print(direct)
#     direct = list(data_ticker.loc[date_pos, 'Direct'])[0]
    if direct == 'Long' and  data.loc[date, 'Low']  < data.loc[start:end, 'Open'].min():
        data_actual_units, id_pos = open_position_long(stocks_dict_add,
                                                      usd_stocks,
                                                      data_actual_units, 
                                                      columns, 
                                                      deposit, 
                                                      date, 
                                                      ticker, 
                                                      period, 
                                                      corr_unit_size, 
                                                      test_list, 
                                                      id_pos, 
                                                      'Long', 
                                                      N,
                                                      data,
                                                      mmd_period, 
                                                      count_ticker_corr) #кол-во открытых тикеров
        return data_actual_units, id_pos
    elif direct == 'Short' and data.loc[date, 'Close'] > data.loc[start:end, 'Open'].max():
        data_actual_units, id_pos = open_position_long(stocks_dict_add,
                                                      usd_stocks,
                                                      data_actual_units, 
                                                      columns, 
                                                      deposit, 
                                                      date, 
                                                      ticker, 
                                                      period, 
                                                      corr_unit_size, 
                                                      test_list, 
                                                      id_pos, 
                                                      'Short', 
                                                      N,
                                                      data,
                                                      mmd_period, 
                                                      count_ticker_corr) #кол-во открытых тикеров
        return data_actual_units, id_pos
    else:
        return data_actual_units, id_pos

### Проверка условия открытия

In [ ]:
def check_open_positions(data_actual_units, 
                         stocks_dict_add,
                         data_stocks_info,
                         data,
                         ticker, 
                         direct, 
                         date,
                         count_direct,
                         count_sector, 
                         count_all, 
                         delta_N_add, 
                         N, 
                         data_ticker,
                         count_ticker, # Колличество юнитов данного тикера
                         n_range_min, # Нижний диапазон N при открытии сделки 
                         n_range_max # Верхний диапазон N при открытии сделки 
                         ):
    """ Функция считает сколько открыто позиций LONG, SHORT и по каждой отрасли
        И есть ли превышение на k*N над уже открытое позицией """
#     data_ticker = data_actual_units[data_actual_units.loc[:,'Ticker'] == ticker] # датафрейм тикера из общего датафрейма открытых позиций
    sector = data_stocks_info[data_stocks_info.loc[:,'Ticker'] == ticker]['Sector'].unique()[0]
### Список цен по открытым юнитам для каждого тикера     
#     last_unit_price_list = list(itertools.chain.from_iterable(list(map(lambda x: list(x.values()), last_unit_price_list))))
        
    count_direct_now = data_actual_units[data_actual_units.loc[:,'Direct'] == direct]['Direct'].count() 
    count_sector_now = data_actual_units[data_actual_units.loc[:,'Sector'] == sector]['Sector'].count()
    count_ticker_now = data_actual_units[data_actual_units.loc[:,'Ticker'] == ticker]['Ticker'].count()
    
    start = date - timedelta(days=5)
    end = date - timedelta(days=3)
    len_units = len(data_actual_units) + 1 #количество открытых юнитов
    
    
#   Блок для фильтрации добавления юнита
#     data = stocks_dict_add[ticker] # Датафрейм с котировками акции
    try: 
        price_now_long = data.loc[date,'Open'] # Цена на данный момент
        price_now_short = data.loc[date, 'Open']
#         price_open = 
        
        if len(data_ticker) > 0: # Если есть открытые позиции
            max_index = data_ticker.index.max()
            price_open = data_ticker.loc[max_index,'Date/price_open'][max_index] # Цена открытия последнего открытого юнита
#             print(max_index, [price_open], '***')
#             price_2N_long = price_open + delta_N_add * N # Цена, на которой должен быть открыт дополнительный юнит 
#             price_2N_short = price_open - delta_N_add * N # Цена, на которой должен быть открыт дополнительный юнит 
            count_ticker_corr = count_ticker_now #кол-во открытых тикеров
        else:
#             price_2N_long = 0 
#             price_2N_short = price_now_short + 1 # Если нет открытых юнитов, то нынешняя цена должна быть быше чем price_2N
#             price_open = 1 if direct=='Long' else -1
            count_ticker_corr = 1
#         print(date, ticker, price_open, price_now)    
        if (direct == 'Long'
            and count_direct_now < count_direct # Позиций в одном направлении не больше 12
            and count_sector_now < count_sector # Позиций в одном секторе не больше 5
            and len_units < count_all # Всего позиций не больше 25
            and count_ticker_now < count_ticker # Юнитов по данному тикеру
#             and price_now_long > price_2N_long   #Если сейчас цена превышает цену price_2N
#             and n_range_min < N < n_range_max
            and price_now_short > data.loc[date, 'SMA'] # Если цена выше средней
#             and data.loc[date, 'SMA'] > data.loc[start:end, 'SMA'].min() # Восходящий тренд
            
           ): 
            
            return 1, count_ticker_corr
        
        elif (direct == 'Short'
            and count_direct_now < count_direct # Позиций в одном направлении не больше 12
            and count_sector_now < count_sector # Позиций в одном секторе не больше 5
            and len(data_actual_units) < count_all # Всего позиций не больше 25
            and count_ticker_now < count_ticker # Юнитов по данному тикеру
#             and price_now_short < price_2N_short  #Если сейчас цена  не превышает цену price_2N
#             and n_range_min < N < n_range_max
            and price_now_long < data.loc[date, 'SMA'] # Если цена ниже средней
#             and data.loc[date, 'SMA'] < data.loc[start:end, 'SMA'].min()
             ):
#             and 0.2 > data.loc[date, '%BB']):
#             all_deal_dict[len_units] = round(sum([list(x.values())[0] for x in list(data_pos_test2['Size/money_unit'])]),2)
            return 1, count_ticker_corr
        else:
            
            return 0, count_ticker_corr
    except KeyError:
        print('shit')
    

In [ ]:
def arr_check_open_positions(arr_data_actual_units, 
                             arr_stocks_dict_add,
                             arr_data_stocks_info, 
                             ticker, 
                             direct, 
                             date,
                             count_direct,
                             count_sector, 
                             count_all, 
                             delta_N_add, 
                             N, 
                             arr_data_ticker, 
                             count_ticker):
    """ Функция считает сколько открыто позиций LONG, SHORT и по каждой отрасли
        И есть ли превышение на k*N над уже открытое позицией """
    sector = arr_data_stocks_info[arr_data_stocks_info[:, 1] == ticker][:,19]
    
    count_direct_now = len(arr_data_actual_units[arr_data_actual_units[:, 3] == direct]) # Кол-во позиций открытых по направлению
    count_sector_now = len(arr_data_actual_units[arr_data_actual_units[:, 2] == sector]) # Кол-во позиций открытых в секторе
    count_ticker_now = len(arr_data_actual_units[arr_data_actual_units[:, 1] == ticker]) # Кол-во позиций открытых в секторе

#   Блок для фильтрации добавления юнита
    arr_data = arr_stocks_dict_add[ticker] # Датафрейм с котировками акции
    price_now = arr_data[arr_data[:, 4] == date][:, 1]
    if len(arr_data_ticker) > 0: # Если есть открытые позиции
        max_index = len(arr_data_ticker)-1
        price_open = arr_data_ticker[max_index, 8]
        price_2N = price_open + delta_N_add * N # Цена, на которой должен быть открыт дополнительный юнит 
#         price_open = data_ticker.loc[max_index,'Date/price_open'][max_index] # Цена открытия последнего открытого юнита
    else:
        price_2N = 0 

    if (count_direct_now < count_direct # Позиций в одном направлении не больше 12
        and count_sector_now < count_sector # Позиций в одном секторе не больше 5
        and count_ticker_now < count_ticker # Юнитов по данному тикеру
        and len(arr_data_actual_units) < count_all # Всего позиций не больше 25
        and price_now > price_2N  #Если сейчас цена превышает цену price_2N
        ):
        return 1

## Закрытие позиций

### Закрытие опциона

In [ ]:
def close_option(data_actual_units, 
                 data_all_units,
                 stocks_dict_add, 
                 data_stocks_info,
                 ticker, 
                 date, 
                 min_max_period, 
                 N, 
                 data_ticker, 
                 data, 
                 i): # i - id позиции
    """Функция закрывает опционную позицию по истечении 21 дня"""
    date_pos = data_ticker.loc[ data_ticker['Id'] == i, 'Date/price_open'].index # Дата открытия уже открытой позиции
    id_pos = data_ticker.loc[date_pos, 'Id'][0]
    N = data_ticker[data_ticker.loc[:,'Id'] == i]['N'][0]
    direct = list(data_ticker.loc[date_pos, 'Direct'])[0]
    size_unit = list(data_ticker.loc[ data_ticker['Id'] == id_pos, 'Size/money_unit'][0].keys())[0]
    price_open = list(data_ticker.loc[ data_ticker['Id'] == id_pos, 'Date/price_open'][0].values())[0]
    expriration_date = date_pos + timedelta(days=35) 
    price_now = data.loc[date, 'Close']
    
    
    price_option_call = 1.17 * price_open  # цена после которой опционы CALL становится прибыльным
    price_option_put = price_open - 0.17 * price_open  # цена после которой опционы PUT становится прибыльным
    # x - price_open: y - price_now; z - price_option_call
    # цена открытия, если цена открытия цена опциона меньше разницы нынешней цены и цены самоокупаемости
    call_option_profit = lambda x, y, z: y-x if y-x < 0.17*x else 0.17*x
    put_option_profit = lambda x, y, z: y-x if y-x < 0.17*x else 0.17*x
#     call_option_profit = lambda x, y, z: x if 0.17*x*(-1) < y - z < 0 else 0.17*x*(-1)
#     put_option_profit = lambda x, y, z: x if 0.17*x*(-1) < z - y < 0 else 0.17*x*(-1)
    
    if (direct == 'Long' 
        and date == expriration_date):  # цена опциона 0.05 цены открытия
        profit = price_now - price_option_call if price_now >= price_open else call_option_profit(price_open,
                                                                                                             price_now,
                                                                                                             price_option_call)# 0.07*price_open*(-1)
        
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = 'Option Call'
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(profit, 2) * size_unit
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(price_now, 2)}]
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'IND'] = data.loc[date, 'IND'] 

        data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
        data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos] 
        return data_actual_units, data_all_units
    elif (direct == 'Short' 
          and date == expriration_date):  # цена опциона 0.05 цены открытия
        profit = price_now - price_option_put if price_now <= price_open else put_option_profit(price_open,
                                                                                                        price_now,
                                                                                                        price_option_put)# 0.07*price_open*(-1)
       
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = 'Option Put'
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(profit, 2) * size_unit
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(price_now, 2)}]
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'IND'] = data.loc[date, 'IND']

        data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
        data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos]         
        return data_actual_units, data_all_units
    else:
        return data_actual_units, data_all_units

### Стоп-лосс

In [ ]:
def close_stop_loss(data_actual_units, 
                    data_all_units,
                    stocks_dict_add, 
                    data_stocks_info,
                    ticker, 
                    date,
                    corr_stop_loss, 
                    N, 
                    data_ticker,  # датафрейм тикера из общего датафрейма открытых позиций
                    data, # ДФ со значениями котировок (уже добавленных)
                    i # Дата
                   ):
    
    """ Функция закрывает позицию если цена ниже/выше цены стоп-лосса """
    date_pos = data_ticker.loc[ data_ticker['Id'] == i, 'Date/price_open'].index # Дата открытия уже открытой позиции
    N = data_ticker[data_ticker.loc[:,'Id'] == i]['N'][0]
    direct = list(data_ticker.loc[date_pos, 'Direct'])[0]
#     direct = data_ticker.loc[i, 'Direct']
#     direct = data_actual_units.loc[data_actual_units['Id'] == id_pos[['Direct'][0]
    id_pos = i #  data_ticker.loc[i, 'Id']
    size_unit = list(data_ticker.loc[ data_ticker['Id'] == id_pos, 'Size/money_unit'][0].keys())[0]
    price_open = list(data_ticker.loc[ data_ticker['Id'] == id_pos, 'Date/price_open'][0].values())[0]
    stop_loss_long = round(price_open - corr_stop_loss*N, 2) # цена стоп-лосса LONG
    stop_loss_short = round(price_open + corr_stop_loss*N, 2) # цена стоп-лосса SHORT
    
#     data.loc[date, 'IND']
    if  direct == 'Long' and data.loc[date, 'Low'] < stop_loss_long:
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(stop_loss_long - price_open, 2) * size_unit
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(stop_loss_long, 2)}]
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = 'Stop loss'
#         data_actual_units.loc[data_actual_units['Id'] == id_pos, 'IND'] = data.loc[date, 'IND'] #/ round(stop_loss_long, 2)
        
        data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
        data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos] # Удаляем строку, если сработал стоп-лосс
#                 data_actual_units = data_actual_units.drop(index = i) 
        
        return data_actual_units, data_all_units

    elif direct == 'Short' and data.loc[date, 'High'] > stop_loss_short: # Если наивысшая сегодняшняя цена больше стоплосса
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(price_open - stop_loss_short, 2) * size_unit
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(stop_loss_short, 2)}]
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = 'Stop loss'
#         data_actual_units.loc[data_actual_units['Id'] == id_pos, 'IND'] = data.loc[date, 'IND']# round(stop_loss_short, 2) / data.loc[date, 'IND']
        
        data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
        data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos] # Удаляем строку, если сработал стоп-лосс
            
        return data_actual_units, data_all_units
    else:
        return data_actual_units, data_all_units
    
def arr_close_stoploss(arr_data_actual_units, # общий массив открытых позиций
                       arr_data_all_units, # Общий массив закрытых позиций
                       arr_data_stocks_dict_add, # Словарь, в который добавляются котировки к конкретным акциям
                       arr_data_stocks_info, # Информация по акциям (справочная)
                       ticker, 
                       date,
                       corr_stop_loss, 
                       N, 
                       arr_data_ticker, # Массив акции с открытыми по ней позиции
                       arr_data, # array со значениями котировок (уже добавленных)
                       i # Строка матрицы
                       ):    
    
    direct = arr_data_actual_units[arr_data_actual_units[:, 7] == i][:,3] # Long=1, Short=-1
    id_pos = arr_data_actual_units[arr_data_actual_units[:, 7] == i][:,0] # Id позиции   
    size_unit = arr_data_actual_units[arr_data_actual_units[:, 7] == i][:,5] # Размер юнита
    price_open = arr_data_actual_units[arr_data_actual_units[:, 7] == i][:,8][0] # Цена открытия уже открытого юнита
    stop_loss_long = round(price_open - corr_stop_loss*N, 2) # цена стоп-лосса LONG
    stop_loss_short = round(price_open + corr_stop_loss*N, 2) # цена стоп-лосса SHORT
    
    if  direct == 1 and arr_data[arr_data[:, 4] == date][:, 3] < stop_loss_long:   # Если минимум определенной даты меньше стоп_лосса 
        # Записываем profit к определенному id
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 12] = round(stop_loss_long - price_open, 2) * size_unit 
        # Записываем дату и цену закрытия
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 9] = date # Дата закрытия
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 10] = round(stop_loss_long, 2) # Цена закрытия
        # Критерий закрытия
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 11] = bins_str('Stop loss')
        # Добавляем полученную строку с id закрытой сделки в общий arr закрытых сделок
        arr_data_all_units = np.vstack((arr_data_all_units, arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos]))
        arr_data_all_units = arr_data_all_units[arr_data_all_units[:0] != id_pos] # Удаляем строку, если сработал стоп-лосс
        return arr_data_actual_units, arr_data_all_units
    
    elif direct == -1 and arr_data[arr_data[:, 4] == date][:, 2] > stop_loss_long:
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 12] =round(price_open - stop_loss_short, 2) * size_unit
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 9] = date # Дата закрытия
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 10] = round(stop_loss_short, 2) # Цена закрытия
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 11] = bins_str('Stop loss')
        arr_data_all_units = np.vstack((arr_data_all_units, arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos]))
        arr_data_all_units = arr_data_all_units[arr_data_all_units[:0] != id_pos] # Удаляем строку, если сработал стоп-лосс
        return arr_data_actual_units, arr_data_all_units

    else:
        return arr_data_actual_units, arr_data_all_units
    

### 2N

In [ ]:
def close_2N(data_actual_units, 
             data_all_units,
             data_ticker,
             data, # ДФ со значениями котировок (уже добавленных)
             N,
             date,
             dev_max_long
             ):

    """ Функция закрывает позицию, если цена идет на 2N от максимума/минимума"""
    id_pos_list = data_ticker.loc[:, 'Id']
#         date_id_pos = data_ticker[data_ticker.loc[:, 'Id'] == id_pos].index
#         print(date_id_pos)
        
    for id_pos in id_pos_list: # Для каждой открытой позиции
        direct = data_ticker[data_ticker.loc[:, 'Id'] == id_pos]['Direct'][0]
        size_unit = list(data_ticker[data_ticker.loc[:, 'Id'] == id_pos]['Size/money_unit'][0].keys())[0] #.apply(lambda x: value(x.keys())))
        price_open = list(data_ticker[data_ticker.loc[:, 'Id'] == id_pos]['Date/price_open'][0].values())[0]
       
        if (direct == 'Long'  
            and data.loc[date, 'Low'] < data.loc[date, 'High'] - dev_max_long * N): # тут может быть дожи и просто очень большая свеча
            N_2_long = data.loc[date, 'High'] - dev_max_long * N # Цена отклонение от максимума
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = "2N"
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(N_2_long - price_open, 2) * size_unit
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(N_2_long, 2)}]
#             data_actual_units.loc[data_actual_units['Id'] == id_pos, 'IND'] = data.loc[date, 'IND'] #/ round(N_2_long, 2)
            
            data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
            data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos] # Удаляем строку, если сработал стоп-лосс

            return data_actual_units, data_all_units

        elif (direct == 'Short' 
#             and len(data_ticker) > 0 #and (date in list(data_ticker.index))
            and data.loc[date, 'Low'] > data.loc[date, 'Low'] + dev_max_long * N): # тут может быть дожи и просто очень большая свеча
            N_2_short =  data.loc[date, 'Low'] + dev_max_long * N # Цена отклонение от максимума
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = "2N"
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(price_open - N_2_short - 0.1, 2) * size_unit
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(N_2_short, 2)}]
#             data_actual_units.loc[data_actual_units['Id'] == id_pos, 'IND'] = data.loc[date, 'IND'] #round(N_2_short, 2) / data.loc[date, 'IND']
            
            data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
            data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos] # Удаляем строку, если сработал стоп-лосс

            return data_actual_units, data_all_units
    return data_actual_units, data_all_units # возврат в случае, если стоплось до этого сделки закрыл 

In [ ]:
        

def arr_close_2N(arr_data_actual_units, 
                 arr_data_all_units,
                 arr_stocks_dict_add, 
                 arr_data_stocks_info,
                 ticker, 
                 date,
                 dev_max_long, 
                 N, 
                 arr_data_ticker, 
                 arr_data, 
                 i):
    direct = i[3] # Long=1, Short=-1
    id_pos = i[0] # Id позиции
    size_unit = i[5] # Размер юнита
    price_open = i[8] # Цена открытия уже открытого юнита
    if  direct == 1 and arr_data[arr_data[:, 4] == date][:, 1] < arr_data[arr_data[:, 4] == date][:, 2] - dev_max_long * N:   # Тут закрываем позицию
        N_2_long = arr_data[arr_data[:, 4] == date][:, 2] - dev_max_long * N
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 11] = bins_str('N') # Criterion
        # Записываем дату и цену закрытия
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 9] = date # Дата закрытия
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 10] = round(N_2_long - 0.1, 2) # Цена закрытия (0.1 - имитация проскальзывния)
        # Profit
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 12] =  round(N_2_long - 0.1 - price_open, 2) * size_unit # Profit
        # Добавляем полученную строку с id закрытой сделки в общий arr закрытых сделок
        arr_data_all_units = np.vstack((arr_data_all_units, arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos]))
        arr_data_all_units = arr_data_all_units[arr_data_all_units[:0] != id_pos] # Удаляем строку, если сработал N
        return data_actual_units, data_all_units    

    elif  direct == -1 and arr_data[arr_data[:, 4] == date][:, 1] > arr_data[arr_data[:, 4] == date][:, 3] + dev_max_long * N:
        N_2_short = arr_data[arr_data[:, 4] == date][:, 3] + dev_max_long * N
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 11] = bins_str('N') # Criterion
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 9] = date # Дата закрытия
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 10] = round(N_2_short+0.1, 2) # Цена закрытия
        arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 12] =  round(price_open - N_2_short - 0.1, 2) * size_unit # Profit
        arr_data_all_units = np.vstack((arr_data_all_units, arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos])) # Добавляем к общиму массиву закрытых позиций
        arr_data_all_units = arr_data_all_units[arr_data_all_units[:0] != id_pos] # Удаляем строку, если сработал N
        return data_actual_units, data_all_units        
    else:
        return data_actual_units, data_all_units    

In [ ]:
#             if direct == 'Long' and data.loc[date, 'Close'] < (data.loc[date, 'High'] - dev_max_long * N): # Тут закрываем позицию
#                 N_2_long = data.loc[date, 'High'] - dev_max_long * N 
    #             print(date, ticker, N_2_long)
#                 data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = "2N"
#                 data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(N_2_long - price_open, 2) * size_unit
#                 data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(N_2_long-0.1, 2)}]
#                 data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
#                 data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos] # Удаляем строку, если сработал стоп-лосс
#                 return data_actual_units, data_all_units

#             elif direct == 'Short' and data.loc[date, 'Close'] > data.loc[date, 'Low'] + dev_max_long * N:
#                 N_2_short =  data.loc[date, 'Low'] + dev_max_long * N
#                 data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = "2N"
#                 data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(price_open - N_2_short - 0.1, 2) * size_unit
#                 data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(N_2_short+0.1, 2)}]
#                 data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
#                 data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos] # Удаляем строку, если сработал стоп-лосс
#                 return data_actual_units, data_all_units
#             return data_actual_units, data_all_units
#         else:
#             return data_actual_units, data_all_units

### Закрытие по миниму n-дней

In [ ]:
def close_profit(data_actual_units, 
                 data_all_units,
                 stocks_dict_add, 
                 data_stocks_info,
                 ticker, 
                 date, 
                 min_max_period, 
                 N, 
                 data_ticker, 
                 data, 
                 i): # i - id позиции
    
    """ Функция закрывает позиции, если прошло минимум "min_max_period" дней со дня  открытия сделки и цена достигла минимума
        этих "min_max_period" дней """
    date_open = list(data_ticker.loc[ data_ticker['Id'] == i, 'Date/price_open'].index )[0]# Дата открытия уже открытой позиции
    date_pos = data_ticker.loc[ data_ticker['Id'] == i, 'Date/price_open'].index
    
    N = data_ticker[data_ticker.loc[:,'Id'] == i]['N'][0]
    if (date-date_pos).days > 0: # Если прошло больше min_max_period дней с даты открытия позиции
        direct = list(data_ticker.loc[date_pos, 'Direct'])[0]
        id_pos = data_ticker.loc[date_pos, 'Id'][0]
        
        start = date - timedelta(days=min_max_period)  # Начало периода
#         while start not in data.index:
#             start = start - timedelta(days=1)     
#         end = date - timedelta(days=1)  # Конец периода
            
        sma = data.loc[date, 'SMA'] #data.loc[start : end, 'Low'].min()  # цена, минимум 10-ти дней
        sma_period = data.loc[start:date, 'SMA']
#         sma = data.loc[start : end, 'High'].max() # цена, максимум 10-ти дней  
        
        size_unit = list(data_ticker.loc[ data_ticker['Id'] == id_pos, 'Size/money_unit'][0].keys())[0]
        price_open = list(data_ticker.loc[ data_ticker['Id'] == id_pos, 'Date/price_open'][0].values())[0]
        if ((direct == 'Long' and data.loc[date, 'Low'] < sma*0.95)
#            or (direct == 'Long' and data.loc[start:date, 'Low'].min() < sma*0.98)
           ): # Если сегодняшняя ниже SMA
            
#             and max_price > min_price + 1*N ): # И если сегодняшняя цена меньше максимума n дней на N отклонений

#             size_unit = list(data_ticker.loc[ data_ticker['Id'] == id_pos, 'Size/money_unit'][0].keys())[0]
#             size_unit = list(data_ticker.loc[i, 'Size/money_unit'].keys())[0]
#             price_open = data_ticker.loc[i,'Date/price_open'][i] # Цена открытия уже открытого юнита

            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = 'min of 10'#.format(min_price, start, end)
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(sma*0.95 - price_open, 2) * size_unit
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(sma*0.95, 2)}]
            
            data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
            data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos] 

            return data_actual_units, data_all_units
        
        elif ((direct == 'Long' and len(data.loc[date_open:date, 'Low']) >= min_max_period
              and data.loc[date, 'Low'] < sma*0.95)
              or (direct == 'Long' and len(data.loc[date_open:date, 'Low']) < min_max_period 
                   and data.loc[date, 'Low'] < sma*0.95)
           ): # Если сегодняшняя ниже SMA
            
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = 'min of period'#.format(min_price, start, end)
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(sma*0.95 - price_open, 2) * size_unit
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(sma*0.95, 2)}]
            
            data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
            data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos] 

            return data_actual_units, data_all_units
        
        
        
        
        
        elif (direct == 'Short' and data.loc[date, 'High'] > sma*1.05):
#             print(i, date, data.loc[date, 'High'], sma*1.08)
#               and max_price - 1*N > min_price  ):
#             size_unit = list(data_ticker.loc[i, 'Size/money_unit'].keys())[0]
#             price_open = data_ticker.loc[i,'Date/price_open'][i] # Цена открытия уже открытого юнита
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = 'min of 10'
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(price_open - sma*1.05, 2) * size_unit
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(sma*1.05, 2)}]
#             data_actual_units.loc[data_actual_units['Id'] == id_pos, 'IND'] = data.loc[date, 'IND'] #round(max_price, 2) / data.loc[date, 'IND']
            
            data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
            data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos]
            return data_actual_units, data_all_units

#         elif (direct == 'Short' and data.loc[start:date, 'High'].max() > sma*1.02):
#        НАДО ВЫЧИЛСЯТЬ МАКСИМАЛЬНУЮ РАЗНИЦУ МЕЖДУ МАКСИИМУМОМ И HIGH
        elif ((direct == 'Short' and len(data.loc[date_open:date, 'High']) >= min_max_period
                  and data.loc[date, 'High'] > sma*1.05)
              or (direct == 'Short' and len(data.loc[date_open:date, 'High']) < min_max_period 
                  and data.loc[date, 'High'] > sma*1.05)
             ):
              
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = 'min of period'
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round(price_open - sma*1.05, 2) * size_unit
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(sma*1.05, 2)}]
#             data_actual_units.loc[data_actual_units['Id'] == id_pos, 'IND'] = data.loc[date, 'IND'] #round(max_price, 2) / data.loc[date, 'IND']
            
            data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
            data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos]
            return data_actual_units, data_all_units
        
        else:
            return data_actual_units, data_all_units
    else:
        return data_actual_units, data_all_units  

In [52]:
    
def arr_close_profit(arr_data_actual_units, 
                     arr_data_all_units,
                     arr_stocks_dict_add, 
                     arr_data_stocks_info,
                     ticker, 
                     date, 
                     min_max_period, 
                     N, 
                     arr_data_ticker, 
                     arr_data, 
                     i):
    if (date-i).days > min_max_period: # Если прошло больше 10 дней с даты открытия позиции

        direct = i[3] # Long=1, Short=-1
        id_pos = i[0] # Id позиции
        size_unit = i[5] # Размер юнита
        price_open = i[8] # Цена открытия уже открытого юнита
        size_unit = i[5] # Размер юнита
        price_open = i[8] # Цена открытия уже открытого юнита
        min_price = min(arr_data[(arr_data[:, 4] > start) & (arr_data[:, 4] < end)][:, 3]) # цена, минимум 10-ти дней
        max_price = max(arr_data[(arr_data[:, 4] > start) & (arr_data[:, 4] < end)][:, 2]) # цена, максимум 10-ти дней 
        if direct == 1 and arr_data[arr_data[:, 4] == date][:, 3] < min_price: # Если сегодняшняя цена меньше минимума n дней
            arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 11] = bins_str('min of 10') # Criterion
            arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 12] = round(min_price-0.5*N - price_open, 2) * size_unit # Profit
            # Записываем дату и цену закрытия
            arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 9] = date # Дата закрытия
            arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 10] = round(min_price - 0.5*N, 2) # Цена закрытия
            # Добавляем полученную строку с id закрытой сделки в общий arr закрытых сделок
            arr_data_all_units = np.vstack((arr_data_all_units, arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos]))
            arr_data_all_units = arr_data_all_units[arr_data_all_units[:0] != id_pos] # Удаляем строку, если сработал стоп-лосс
            return data_actual_units, data_all_units
    
        elif direct == -1 and arr_data[arr_data[:, 4] == date][:, 2] > max_price:
            arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 11] = bins_str('min of 10') # Criterion
            arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 12] = round(price_open - max_price+0.5*N, 2) * size_unit
            arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 9] = date
            arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos][:, 10] = round(max_price+0.5*N, 2)
            arr_data_all_units = np.vstack((arr_data_all_units, arr_data_actual_units[arr_data_actual_units[:, 0] == id_pos]))
            arr_data_all_units = arr_data_all_units[arr_data_all_units[:0] != id_pos] # Удаляем строку, если сработал стоп-лосс
            return data_actual_units, data_all_units            
                    
    else:
        return data_actual_units, data_all_units

In [ ]:
def min_price(data, date, min_max_period):
    """ Функция  вычисляет наименьшую цену за опредленный период свечей, учитываю числа, в которых не было торгов"""
    date_quotes = data.loc[date, :]
    data = data.reset_index(drop=True)
    start_index = data[data.loc[:,:] == date_quotes].index #- 5
    print(start_index)
#     return data.loc[]
#     min_price = data.loc[start : end, 'Low'].min()
    
    

### Принудительное закрытие сделок

In [ ]:
def close_all_deals(data_actual_units, 
                    data_all_units,
                    stocks_dict_add, 
                    date):
    """Функция принудительно закрывает все сделки по окончанию периода"""
    for id_pos in list(data_actual_units['Id']):
        ticker = list(data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Ticker'])[0]
        data = stocks_dict_add[ticker]
        price_open = list(data_actual_units[data_actual_units.loc[:, 'Id'] == id_pos]['Date/price_open'][0].values())[0] 
        date = data.index.max()
        direct = data_actual_units[data_actual_units.loc[:, 'Id'] == id_pos]['Direct'][0]

            
        size_unit = list(data_actual_units.loc[ data_actual_units['Id'] == id_pos, 'Size/money_unit'][0].keys())[0]
        price_close = data.loc[date, 'Close']

        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Criterion'] = 'out of per'#.format(min_price, start, end)
        if direct == 'Long':
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round((price_close - price_open) * size_unit,2)
        elif direct == 'Short':
            data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Profit'] = round((price_open - price_close) * size_unit,2)
            
        data_actual_units.loc[data_actual_units['Id'] == id_pos, 'Date/price_close'] = [{date : round(price_close, 2)}]
    #         print(ticker)

        data_all_units = pd.concat([data_all_units, data_actual_units.loc[data_actual_units['Id'] == id_pos]])
        data_actual_units = data_actual_units.loc[data_actual_units['Id'] != id_pos] 


    return data_actual_units, data_all_units

### Прочие функции для работы алгоритма

In [28]:
def sum_count_deals(all_deal_dict):
    dict0 = {1:2, 3:4, 5:6}
    count = max(list(all_deal_dict.keys()))
#     print(count, all_deal_dict)
    summa = all_deal_dict[count]
    return summa, count
                


6


In [ ]:
def N_mean(data, # ДФ со значениями котировок (уже добавленных)
           date):
    """ Функция вычисляет значение N для расчета отклонения цены при закрытии сделки. """
    
    start = date - timedelta(days=1)
#     N_end = max_index - timedelta(days=1)
#     N = data.loc[N_start : N_end, 'N'].mean()
    N = data.loc[start, 'N']
#     print(data)
    return N

def arr_N_mean(arr_data,
                 date):
    N = arr_data[arr_data[:, 4] == date][:, 7][0]
    return N

In [ ]:
ASCII_BITS = 8
def to_bin(c):
    """ Функция кодирует строку в двоичный код"""
    return bin(ord(c))[2:].zfill(ASCII_BITS)
def bins_str(text):
    """ Функция переводит двоичный код в десятичный"""
    bin_str = ''.join(to_bin(c) for c in text) # Строка в бинарном представлении
    return int(bin_str, 2) # Бинарное число переводится в десятичное
print(bins_str('Hi'))

In [ ]:
def int_to_text(number):
    """ Функция переводит число в слово"""
    bin_str = bin(number)[2:]
    pad = '0' * (ASCII_BITS - len(bin_str) % ASCII_BITS)
    bin_str = pad + bin_str
    
    chars = []
    for i in range(0, len(bin_str), ASCII_BITS):
        bin_part = bin_str[i: i + ASCII_BITS]
        c = chr(int(bin_part, 2))
        chars.append(c)

    return ''.join(chars)



In [ ]:
# # ПРЕДПОДГОТОВКА. Чтобы не запускать алгоритм на повторяющихся действиях
# date_from = datetime.datetime(2021, 1, 5)
# date_to = datetime.datetime(2021, 4, 30)
# usd_stocks, rub_stocks = rub_usd(data_stocks)
# stocks_list = list(usd_stocks.loc[0:40,'Ticker'])#.sample(frac=1)) #Список интересующих акций
# stocks_dict_full = create_stocks_dict_full(stocks_list, usd_stocks, date_from, date_to) #создание словаря с выбранными акциями

## Алгоритм

### array

In [ ]:
def arr_open_turtles(
                 usd_stocks, # Вынесено за пределы функции
                 stocks_list, # Вынесено за пределы функции
                 stocks_dict_full, # Вынесено за пределы функции
                 deposit, 
                 interval_start, # Начало списка тикеров
                 interval_end, # Конец списка тикеров
                 corr_stop_loss, # На сколько отодвигается стоп-лосс
                 corr_unit_size,
                 add_corr_N,
                 count_direct,
                 count_sector, 
                 count_all,
                 count_ticker,
                 dev_max_long, # Отклонение цены от максимума
                 min_max_period, # Закрытие на минимуме или максимуме за определенный период
                 delta_N_add, # На сколько N должна повыситься/понизиться цена, чтобы добавить юнит
                 year_from, # Год начала периода
                 year_to, # Год конца периода
                 period # Период, за который ищется предыдущий прорыв 
                    ): 
    """ Функция прогоняет список акций в виде np.array-массива, а не DataFrame."""
    date_from = datetime.datetime(year_from, 2, 5)
    date_to = datetime.datetime(year_to, 12, 30)
    
    period_small = 20
    period_big = 55
    
    arr_columns = ['Id', 'Ticker', 'Sector','Direct', 'Lot', 'Size_unit', 'Money_unit', 'Date_open', 'Price_open', 'Date_close', 'Price_close', 
                     'Criterion', 'Profit', 'ATR', 'N']
    arr_data_actual_units = np.empty((0,15), int) # матрица с открытыми позициями

    arr_data_all_units = np.empty((0,15), int) # матрица со всеми позициями (каждый массив из 15 значений)
    
    arr_data_stocks_dict_add = arr_create_stocks_dict_add(stocks_list) # Сюда будут добавляться котировки
    
    calendar = calendar_year(date_from, date_to) # Календарь
    
    parallel_deals = [] # Колличество одновременно открытых сделок
    
    test_list = []
    test_list_2 = []
    id_pos = 0 # Id сделки
    for date in tqdm(calendar):
        for ticker in stocks_list:
            arr_data_ticker = arr_data_actual_units[arr_data_actual_units[:, 1] == ticker]  # array тикера из общего датафрейма открытых позиций
#             print (arr_data_ticker)
                
            if (date.weekday() in [0,1,2,3,4]): # Проверка что день - будний и есть открытые сделки по тикеру
                
                try: 
                    arr_all_quotes_stock = stocks_dict_full[ticker] # array со всеми котировками по конкретному тикеру (с 2000 года)
                    arr_data_stocks_dict_add[ticker] = np.vstack((arr_data_stocks_dict_add[ticker], arr_all_quotes_stock[arr_all_quotes_stock[:, 4] == date])) #Добавили строку в array по тикеру в словарь котировок
                    
                    N = arr_N_mean(arr_data_stocks_dict_add[ticker], date) # вычисляем N
                    arr_data = arr_data_stocks_dict_add[ticker] # array со значениями котировок (уже добавленных)

                    
                    if len(arr_data_ticker) > 0: # Если есть открытые позиции
#                         arr_data = arr_data_stocks_dict_add[ticker] # array со значениями котировок (уже добавленных)
                               
                        for i in arr_data_ticker[:, 7]: # Для каждого юнита прогоняет все 3 условия (индексация по дате открытых позиций)

                            arr_data_actual_units, arr_data_all_units = arr_close_stoploss(arr_data_actual_units, arr_data_all_units, arr_data_stocks_dict_add, 
                                                                                            arr_usd_stocks, ticker, date, corr_stop_loss,
                                                                                            N, arr_data_ticker, arr_data, i) # Закрывает по стоплоссу
                            arr_data_actual_units, arr_data_all_units = arr_close_profit(arr_data_actual_units, arr_data_all_units, arr_data_stocks_dict_add, 
                                                                             arr_usd_stocks, ticker, date, min_max_period,
                                                                             N, arr_data_ticker, arr_data, i) # Закрывает по минимум 10 дней
#                             arr_data_actual_units, arr_data_all_units = arr_close_2N(arr_data_actual_units, arr_data_all_units, arr_data_stocks_dict_add, 
#                                                                          arr_usd_stocks, ticker, date,corr_unit_size, dev_max_long, 
#                                                                          N, arr_data_ticker, arr_data, i) # Закрывает по отклонению 2N
                    
                    b_long_small = arr_breakout_long(period_small, arr_data_stocks_dict_add, date, ticker)
                    b_short_small = arr_breakout_short(period_small, arr_data_stocks_dict_add, date, ticker)
                except:
                    b_long_small = 0
                    b_short_small = 0
     ###### Если сегодня есть прорыв канала, проверяем предыдущий прорыв на прибыльность. Условие LONG. Период 20 дней
                if (b_long_small == 1 and b_short_small != 1 and N < 100): # Условие, что прорыв только из при небольшом N
                    b_common_p = arr_breakout_common_prev(period, arr_data_stocks_dict_add, date, ticker) # Условие: предыдущтй прорыв был прибыльным
#                     print(date)
                    if (b_common_p == 1 # предыдущий прорыв был прибыльный  
                        and arr_breakout_long(period_big, arr_data_stocks_dict_add, date, ticker) == 1 # Был прорыв в большом периоде
                        and arr_check_open_positions(arr_data_actual_units, arr_data_stocks_dict_add, arr_usd_stocks, ticker, 'Long', date,
                                                     count_direct, count_sector, count_all, delta_N_add, N, arr_data_ticker, 
                                                     count_ticker) == 1):
                        
                        arr_data_actual_units, id_pos = arr_open_position_long(arr_data_stocks_dict_add,
                                                                          arr_usd_stocks, 
                                                                          arr_data_actual_units, 
                                                                          arr_columns,
                                                                          deposit,
                                                                          date,
                                                                          ticker, 
                                                                          period_big, 
                                                                          corr_unit_size, 
                                                                          test_list,
                                                                          id_pos, 
                                                                          'Long',
                                                                          N, 
                                                                          arr_data)
#                         parallel_deals.append(arr_data_actual_units[arr_data_actual_units.loc[:, 'Ticker'] == ticker]['Ticker'].count())
                  
                    
                    elif (b_common_p == 0 # предыдущий прорыв был убыточный  
                          and arr_check_open_positions(arr_data_actual_units, arr_data_stocks_dict_add, arr_usd_stocks, ticker, 'Long', date, 
                                                       count_direct, count_sector, count_all, delta_N_add, N, arr_data_ticker,
                                                       count_ticker) == 1):
                        arr_data_actual_units, id_pos = arr_open_position_long(arr_data_stocks_dict_add,
                                                                              arr_usd_stocks,
                                                                              arr_data_actual_units, 
                                                                              arr_columns,
                                                                              deposit,
                                                                              date,
                                                                              ticker,
                                                                              period_small,
                                                                              corr_unit_size,
                                                                              test_list,
                                                                              id_pos, 
                                                                              'Long', 
                                                                              N, 
                                                                              arr_data)
#                         parallel_deals.append(arr_data_actual_units[arr_data_actual_units.loc[:, 'Ticker'] == ticker]['Ticker'].count())

######## ТУТ НАЧИНАЕТСЯ SHORT      
                elif (b_short_small == 1 and b_long_small != 1 and N < 100):
                    b_common_p = arr_breakout_common_prev(period, arr_stocks_dict_add, date, ticker) # Условие: предыдущтй прорыв был прибыльным
                
                    if (b_common_p == 1 # предыдущий прорыв был прибыльный  
                        and arr_breakout_short(period_big, stocks_dict_add, date, ticker) == 1 # Был прорыв в большом периоде
                        and arr_check_open_positions(arr_data_actual_units, arr_data_stocks_dict_add, arr_usd_stocks, ticker, 'Short', date,
                                                     count_direct, count_sector, count_all, delta_N_add, N, arr_data_ticker,
                                                     count_ticker) == 1):
                        
                        arr_data_actual_units, id_pos = arr_open_position_long(arr_data_stocks_dict_add,
                                                                              arr_usd_stocks, 
                                                                              arr_data_actual_units, 
                                                                              arr_columns,
                                                                              deposit,
                                                                              date,
                                                                              ticker, 
                                                                              period_big, 
                                                                              corr_unit_size, 
                                                                              test_list,
                                                                              id_pos, 
                                                                              'Short', 
                                                                              N, 
                                                                              arr_data)
#                         parallel_deals.append(arr_data_actual_units[arr_data_actual_units.loc[:, 'Ticker'] == ticker]['Ticker'].count())

                    elif (b_common_p == 0 # предыдущий прорыв был убыточный  
                          and arr_check_open_positions(arr_data_actual_units, arr_data_stocks_dict_add, arr_usd_stocks, ticker, 'Short', date, 
                                                       count_direct, count_sector, count_all, delta_N_add, N, arr_data_ticker,
                                                       count_ticker) == 1):
                        arr_data_actual_units, id_pos = arr_open_position_long(arr_data_stocks_dict_add,
                                                                              arr_usd_stocks,
                                                                              arr_data_actual_units, 
                                                                              arr_columns,
                                                                              deposit,
                                                                              date,
                                                                              ticker,
                                                                              period_small,
                                                                              corr_unit_size,
                                                                              test_list,
                                                                              id_pos, 
                                                                              'Short', 
                                                                              N, 
                                                                              arr_data)
#                         parallel_deals.append(arr_data_actual_units[arr_data_actual_units.loc[:, 'Ticker'] == ticker]['Ticker'].count())
                                 
                
    display(arr_data_actual_units)
    data_all_units = pd.DataFrame(data=arr_data_all_units, columns=arr_columns)            
    unique_list = list(set(test_list))
    
#     display ('Все сделки', data_all_units.sort_index())
    print('Уникальных инструментов:{}/{}'.format(len(data_all_units.loc[:,'Ticker'].unique()), len(unique_list)))
    print ('Profit:', data_all_units.loc[:,'Profit'].sum())
#     print ('Максимальное колличество юнитов:', max(parallel_deals))
    
    return data_all_units.sort_values(by='Id'), data_all_units.loc[:,'Profit'].sum() 

### dataframe

In [ ]:
# %%time
# Входящие данные
# date_from = datetime.datetime.strptime('2018/6/13', '%Y/%m/%d').date()
# date_to = datetime.datetime.strptime('2018/8/13', '%Y/%m/%d').date()
# usd_stocks, rub_stocks = rub_usd(data_stocks)
def open_turtles(
                 usd_stocks, # Вынесено за пределы функции
                 stocks_list, # Вынесено за пределы функции
                 stocks_dict_full, # Вынесено за пределы функции
                 deposit, 
                 corr_stop_loss, # На сколько отодвигается стоп-лосс
                 corr_unit_size, # коэф. размера позиции
                 add_corr_N,
                 count_direct, # колличество позиций по одному направлению
                 count_sector, # колличество позиций в секторе
                 count_all, # всего позиций
                 count_ticker, # Кол-во открытых позиций по тикеру
                 dev_max_long, # Отклонение цены от максимума
                 min_max_period, # Закрытие на минимуме или максимуме за определенный период
                 delta_N_add, # На сколько N должна повыситься/понизиться цена, чтобы добавить юнит
                 year_from, # Год начала периода
                 year_to, # Год окончания периода
                 period, # Период, за который ищется предыдущий ОБЩИЙ прорыв 
                 n_range_min, # Нижний диапазон N при открытии сделки 
                 n_range_max, # Верхний диапазон N при открытии сделки
                 mmd_period, # Период, за который рассчитывается диапазон (min-max) / (min+max)
                 breakout_period, # Период за который ищется прорыв
                 write_to_csv=False): 
    parametrs = dict([('Depo', deposit), 
                 ('Corr_s_l', corr_stop_loss), ('Co_dir', count_direct), ('Co_sector', count_sector), ('Co_all', count_all),
                 ('Co_tickr', count_ticker), ('D_m_long', dev_max_long), ('M_m_per', min_max_period),
                 ('D_n_a', delta_N_add), ('Y_f_t', [year_from, year_to]), ('Mmd_per', mmd_period),
                 ('Brea_per', breakout_period)])# Параметры, которые передаются в таблицу csv для анализа
    
    date_from = datetime.datetime(year_from, 1, 5)
    date_to = datetime.datetime(year_to, 12, 30)
    
    period_small = breakout_period 
    period_big = breakout_period 
    
    arr_all_units = np.empty((0,15), int) # матрица со всеми позициями (каждый массив из 15 значений)
    
    stocks_dict_add = create_stocks_dict_add(stocks_list) # Сюда будут добавляться котировки

    
    calendar = calendar_year(date_from, date_to) # Календарь
    parallel_deals = [] # Количество одновременно открытых сделок по одному тикеру
    all_deal_dict = {} # Количество одновременно открытых позиций и сумма
    test_list = []
    test_list_2 = []
    id_pos = 0 # Id сделки
    ## Добавление индикатора в data
    for i in stocks_dict_full.keys():
#         stocks_dict_full[i] = stocks_dict_full[i].drop(columns='IND3') 
#         stocks_dict_full[i] = stocks_dict_full[i].drop(columns='PV') 
#         stocks_dict_full[i] = stocks_dict_full[i].drop(columns='CV') 
#         stocks_dict_full[i] = stocks_dict_full[i].drop(columns='%BB')
#         stocks_dict_full[i] = stocks_dict_full[i].drop(columns='IND4')
        try:
            stocks_dict_full[i], ind = BB(stocks_dict_full[i])
            stocks_dict_full[i], ind2 = BB2(stocks_dict_full[i])
            stocks_dict_full[i], ind3 = BB3(stocks_dict_full[i])
        except:
            print('Indicator error')
        
    columns = ['Id', 'Ticker', 'Sector','Direct', 'Lot','Size/money_unit', 'Date/price_open', 'Date/price_close', 
               'Criterion', 'Profit', 'ATR', 'N', ind, ind2, ind3]

    data_actual_units = pd.DataFrame(columns=columns) # датафрейм с открытыми позициями
    arr_actual_units = np.empty((0,15), int) # матрица с открытыми позициями
    
    data_all_units =  pd.DataFrame(columns=columns) # датафрейм со всеми позициями
        
    for date in tqdm(calendar):
        for ticker in stocks_list:
            all_deal_dict[len(data_actual_units)+id_pos*0.01] = round(sum([list(x.values())[0] for x in list(data_actual_units['Size/money_unit'])]),2)

            
            data_ticker = data_actual_units[data_actual_units.loc[:,'Ticker'] == ticker] # датафрейм тикера из общего датафрейма открытых позиций
            stocks_dict_add[ticker] = pd.concat([stocks_dict_add[ticker], stocks_dict_full[ticker][stocks_dict_full[ticker].index == date]]) #Добавили строку в словарь котировок       
            data = stocks_dict_add[ticker] # ДФ со значениями котировок (уже добавленных)
            
            
            start = date - timedelta(days=10)
            end = date - timedelta(days=1)
            
            if len(data) > 0 and (date in list(data.index)): # Если есть котировки по акции
                N = data.loc[date, 'N']
                id_list = list(data_ticker.loc[:,'Id'])
#                 price_now = data.loc[date,'Open'] # Цена открытия (ищем дешевые акции)

                if len(data_ticker) > 0 : # Если есть открытые позиции
#                     display(data_ticker)
                    for i in id_list: # Для каждого юнита прогоняет все 3 условия (индексация по дате открытых позиций)
#                         data_actual_units, data_all_units = close_option(data_actual_units, data_all_units, stocks_dict_add, 
#                                                                          usd_stocks, ticker, date, min_max_period,
#                                                                          N, data_ticker, data, i) # Закрывает по минимум 10 дней
            
#                         print(date,i)
                        data_actual_units, data_all_units = close_profit(data_actual_units, data_all_units, stocks_dict_add, 
                                                                         usd_stocks, ticker, date, min_max_period,
                                                                         N, data_ticker, data, i) # Закрывает по минимум 10 дней
                      
#                         data_actual_units, data_all_units = close_stop_loss(data_actual_units, data_all_units, stocks_dict_add, 
#                                                                              usd_stocks, ticker, date, corr_stop_loss,
#                                                                              N, data_ticker, data, i) # Закрывает по стоплоссу

#                     data_actual_units, id_pos = add_unit(usd_stocks, stocks_dict_add, columns,
#                                                              deposit,date, ticker,period,corr_unit_size,test_list,
#                                                              N,mmd_period,count_ticker_corr,data_actual_units,
#                                                              data_all_units, data_ticker, data, id_pos)
#                         parallel_deals.append(data_actual_units[data_actual_units.loc[:, 'Ticker'] == ticker]['Ticker'].count())
                        
                breakout = breakout_common(breakout_period, stocks_dict_add, date, ticker, data)
#                 b_short_small = breakout_short(period_small, stocks_dict_add, date, ticker, data)
#                 breakout = numpy_breakout_common(period, stocks_dict_add,date, ticker, data)
                
                sma = data.loc[date, 'SMA']
     ###### Если сегодня есть прорыв канала, проверяем предыдущий прорыв на прибыльность. Условие LONG. Период 20 дней
                if ((breakout == 1 and len(data_ticker)==0) 
                    or (len(data_ticker.query('Direct == "Long"')) > 0 and  data.loc[date, 'Low']  < data.loc[start:end, 'Low'].min())
                    or (len(data_ticker.query('Direct == "Long"')) > 0 and  data.loc[date, 'Low'] < sma*1.09 < data.loc[start:end, 'Close'].max()-2*N)): 
#                     print(date, data.loc[start:end, 'Close'].max()-2*N)
                    
                    check, count_ticker_corr = check_open_positions(data_actual_units, stocks_dict_add,
                                                                usd_stocks, data, ticker, 'Long', date,
                                                                count_direct, count_sector, count_all, delta_N_add, N, data_ticker, 
                                                                count_ticker, n_range_min, n_range_max) 
#                     print('breakout', date, breakout)
#                     count_ticker_corr =1 
################################# ТУТ НАЧИНАЕТСЯ LONG  
                    if (check == 1):
                        data_actual_units, id_pos = open_position_long(stocks_dict_add,
                                                                      usd_stocks, 
                                                                      data_actual_units, 
                                                                      columns,
                                                                      deposit,
                                                                      date,
                                                                      ticker, 
                                                                      period_big, 
                                                                      corr_unit_size, 
                                                                      test_list,
                                                                      id_pos, 
                                                                      'Long',
                                                                      N,
                                                                      data,
                                                                      mmd_period, 
                                                                      count_ticker_corr)
#                         breakout_common_prev(period, stocks_dict_add, date, ticker) # Расчет прибыльности предыдущего прорыва

                        parallel_deals.append(data_actual_units[data_actual_units.loc[:, 'Ticker'] == ticker]['Ticker'].count())
                   

################################# ТУТ НАЧИНАЕТСЯ SHORT      
                elif ((breakout == 2 and len(data_ticker)==0)
#                     or (len(data_ticker.query('Direct == "Short"')) > 0 and  data.loc[date, 'High']  > data.loc[start:end, 'High'].max()) 
                    or (len(data_ticker.query('Direct == "Short"')) > 0 and  data.loc[date, 'High'] > sma*0.91 > data.loc[start:end, 'Close'].max()+2*N)
                     ):
#                     b_common_p = breakout_common_prev(period, stocks_dict_add, date, ticker) # Условие: предыдущтй прорыв был прибыльным
#                     print(date, len(data_ticker.query('Direct == "Short"')))
                    check, count_ticker_corr = check_open_positions(data_actual_units, stocks_dict_add, usd_stocks, data, ticker, 'Short', date,
                                                             count_direct, count_sector, count_all, delta_N_add, N, data_ticker, 
                                                             count_ticker, n_range_min, n_range_max) 
                   
                
                    if (check == 1):
#                 (breakout_short(period_big, stocks_dict_add, date, ticker, data) == 1 # Был прорыв в большом периоде
#                         and b_common_p == 1  # предыдущий прорыв был прибыльный  
                        data_actual_units, id_pos = open_position_long(stocks_dict_add,
                                                                       usd_stocks, 
                                                                       data_actual_units, 
                                                                       columns,
                                                                       deposit,
                                                                       date,
                                                                       ticker, 
                                                                       period_big, 
                                                                       corr_unit_size, 
                                                                       test_list,
                                                                       id_pos, 
                                                                       'Short', 
                                                                       N,
                                                                       data,
                                                                       mmd_period,
                                                                       count_ticker_corr)
#                         breakout_common_prev(period, stocks_dict_add, date, ticker) # Расчет прибыльности предыдущего прорыва
                        parallel_deals.append(data_actual_units[data_actual_units.loc[:, 'Ticker'] == ticker]['Ticker'].count())
                      
#                     if len(data_ticker) > 0 : # Если есть открытые позиции
#                     for i in id_list: # Для каждого юнита прогоняет все 3 условия (индексация по дате открытых позиций)        
#                         data_actual_units, data_all_units = close_2N(data_actual_units, data_all_units, data_ticker, data, N, date,
#                                                                      dev_max_long) 
#             else:
#                 print('date')
    data_actual_units, data_all_units = close_all_deals(data_actual_units, data_all_units, stocks_dict_add, date)
#     display(data_all_units.loc[:,'Ticker'].unique())
#     print('Тикеры открытых сделок:', list(set(test_list)))
    display('Открытые сделки', data_actual_units)
    unique_list = list(set(test_list))
    print('-------------------------------------')
#     display ('Все сделки', data_all_units.sort_index())
    print('Уникальных инструментов:{}/{}'.format(len(data_all_units.loc[:,'Ticker'].unique()), len(unique_list)))
    print ('Profit:', data_all_units.loc[:,'Profit'].sum())
    print ('Максимальное колличество юнитов:', max(parallel_deals))
    print ('Сумма и колличество одновременно открытых сделок:', sum_count_deals(all_deal_dict))
    
    if write_to_csv == True:
        results_to_csv(data_all_units, parametrs) # Запись в таблицу результатов
#     else:
#         df_result = pd.read_csv(r'C:\Users\ogurskiy\Desktop\DF\results.csv', sep=';') # Показ того, что есть в таблице
#         display(df_result)
    
    return data_all_units.sort_values(by='Id'), data_all_units.loc[:,'Profit'].sum() 

In [ ]:
# display(create_analytical(data_pos))

# Замечания

План на следующий день от 23.03.2022

Готов выход по стоп лоссам. Выходы записываются в отдельный датафрейм data_all_units. Актуальный (не закрытые) сделки находятся в data_actual_units

Сделать: 
- +Выход с прибылью после минимума 10 дней. **** ДОБАВЛЕНО
- +Сделать проверку, чтобы функция close_positions вызывалась только тогда, когда data_actual_units не пустой, и только для тикеров, по которым есть открытые позиции
- Объединить функциии выхода в одну
- +Посмотреть, что там с 2N **** Исправлено 
- +Добавить SHORT **** Добавлено
- +ПОЧЕМУ НЕ ВСЕ ТИКЕРЫ УЧАВСТВУЮТ В СДЕЛКАХ? Возможно датафрем обновляется странно **** ИСПРАВЛЕНО
- +Код падает после изменения даты **** Исправлено
- +Разная прибыль в один промежуток времени **** исправил, дело было в id
- Добавить условие высокой волатильности. Когда канал пробит и вверх и вниз
- +Сделать индексом ДАТА + ВРЕМЯ **** id_pos
- Нужно придумать, чтобы открытые сделки мониторились постоянно
- Алгоритм построен так, что учитывается безлимитное плечо
- +Выкинуть из календаря все дни-выходные **** Цикл по выходным не запускается
- При прочих равных выбирать акции типа VIPS. То есть которые имеют минимальный дневной диапазон 
- +Разобраться с money_unit. Когда зона консолидации небольшая, то очень большой размер позиции **** ИСПРАВЛЕНО
- Написать подсказки и словари для функций ARR
- Сделать FAQ со значениями переменных
- Сделать что-то с внутриканальным открытием
- Сделать разные условия закрытия для большого N и для меньше 1
- Сделать возможным открывать неск юнитов на 1 свече (для этого переделать все по id_pos)
- Протестировать сценарий, при котором 3 разных котировки на одной дате. (Имитация вызова алгоритма много раз за день)
- Маленькое значение N на протяжении нескольких дней вводит в заблуждение, и ведет к открытию больших позиций, в то время как их вышибает стоп лоссом 

- Обучить АМО для пресказывания по показателяем нескольких индикаторов прибыльная или убыточная будет сделка
- Репрезентативность может быть достигнула с помощью меньшего колличества акций, но нескольких периодов
- ИДЕЯ. Увеличить отклонение для стоп лосса, но покупать или продавать опцион на этот инструмент. Большой стоп лосс даст возможность ловить большие тренды, а от просадки будет страховать опцион.
- ИДЕЯ. Колл опционы для хуевых акций, с далеким страйком. Теряем по чуть чуть, но емае если цена будет достигнута
- ИДЕЯ. Рассчитывать N как показатель на тот преиоды, при котором происходит прорыв. То есть за 20 или 55 дней
- ИДЕЯ. Увеличить стоплосс, и посмотреть на сколько увеличится прибыль по прибыльным позициям, то есть без учета убыточных, так как это будет имитация покупкой или продажей опциона. + вычесть премии за опционы (нужно рассчитать)
- ИДЕЯ. Если покупать и продавать делая позицию НЕ дельта нейтральной, то можно попробовать собирать больше прибыль при фиксированных убытках
- ИДЕЯ. Опцион вне денег быстрее дорожает чем опцион на деньгах если возрастает волатильность.. Это актуально для прорывных стратегий. 
- ИДЕЯ. Заменить min_of_10 касанием средней
- ИСПРАВИТЬ. При закрытия на 2N учитывать N предыдущего дня а не дня открытия позиции.
- ИДЕЯ. Считать разброс цен для 10 дней
- ИДЕЯ. Построить график, на котором будут выявлены нанесены все сделки и сделки по отраслям.
- ИДЕЯ. Сочетать длинную позицию на одном рынке с длинной позицией на родственном рынке 
- ПРОВЕРИЛ. ИДЕЯ. Делать выход не по минимуму 10 дней а по касанию средней
- ИДЕЯ. Корреляция объема и прибыльности прорыва
- ИДЕЯ. Если акция после прорыва вернулась в корридор, сокращать позицию на 50%
- ИДЕЯ. В каждый момент времени иметь дешевый стрэдл как защиту от волатильности портфеля
- ИДЕЯ. Рассмотреть корреляцию цены акции и прибыльности прорыва. Гипотеза: дешевые акции имеют больше ложных прорывов
- ИДЕЯ. В кризис и периоды высокой волатильности как в 2020(начало ковида), и в 2022 году(война в Украине) опционные могут принести больше денег
- ИДЕЯ. Опционы в деньгах стоят больше, но для окупаемости они выгоднее, так как быстрее приносят прибыль
- ИДЕЯ. Если изменение цены и соответствующий пробой изменяется вместо с повышением/понижением объема, ставить на направление пробоя.
- ИДЕЯ. Парсить цены на покупки и продажу, для того, чтобы открывать позиции с нулевыми или около нулевыми затратами
- *ХОРОШАЯ ИДЕЯ* Спреды дешевле стренглов, стредлов и тд, при этом, если при высокой волатильности открывать спреды с большой разницей в покупке/продаже, то уменьшается размер потенцйиальных убытков, при это диапазон опциона досточный чтобы перекрыть движение цены 60% (не беру в расчет колебания выше 60%, так как их поймать крайне тяжело и крайне редки, при экспирации опциона +- месяц)
- ИДЕЯ. Одновременно должно работать несколько стратегий с разной логикой, для того, чтобы если одна не сработала, то сработала бы другая. Напрмер торговля по тренду и страхование опционами.
- ИДЕЯ. Открываться вверх, только когда есть средняя с большим периодом имеет восходящий тренд, и на данный момент веча находится сверху средней.
- ИДЕЯ. Проверять предыдущий прорыв всегда, а не только на первой сделке. Исследовать конкретные акции, смотреть какие из них более склонны к ложным прорывам. Должно быть подтверждение объемом. Объем нормализовывать как отшение нынешнего объема к n*K дней до него.
- ИДЕЯ. Добавлять к позиции только на откатах.
- ИДЕЯ. Сформировать списки акций на покупку и на продажу. И сигналы на покупку и продажу применять только к соответствующим спискам.
- ИДЕЯ. Чем больше период прорыва, тем больше позиция
- !! Сделать индикатор максимального прорыва, который показывает максимальный показатель прорыва для сделки. И второе условие - открытие позиции на откате
- Для больших позиций ставить стоп лосс поближе

## Старая версия

In [ ]:
# # %%time
# # Входящие данные
# # date_from = datetime.datetime.strptime('2018/6/13', '%Y/%m/%d').date()
# # date_to = datetime.datetime.strptime('2018/8/13', '%Y/%m/%d').date()
# # usd_stocks, rub_stocks = rub_usd(data_stocks)
# def open_turtles(
#                  usd_stocks, # Вынесено за пределы функции
#                  stocks_list, # Вынесено за пределы функции
#                  stocks_dict_full, # Вынесено за пределы функции
#                  deposit, 
#                  corr_stop_loss, # На сколько отодвигается стоп-лосс
#                  corr_unit_size, # коэф. размера позиции
#                  add_corr_N,
#                  count_direct, # колличество позиций по одному направлению
#                  count_sector, # колличество позиций в секторе
#                  count_all, # всего позиций
#                  count_ticker, # Кол-во открытых позиций по тикеру
#                  dev_max_long, # Отклонение цены от максимума
#                  min_max_period, # Закрытие на минимуме или максимуме за определенный период
#                  delta_N_add, # На сколько N должна повыситься/понизиться цена, чтобы добавить юнит
#                  year_from, # Год начала периода
#                  year_to, # Год окончания периода
#                  period, # Период, за который ищется предыдущий ОБЩИЙ прорыв 
#                  n_range_min, # Нижний диапазон N при открытии сделки 
#                  n_range_max, # Верхний диапазон N при открытии сделки
#                  mmd_period, # Период, за который рассчитывается диапазон (min-max) / (min+max)
#                  breakout_period # Период за который ищется прорыв
#                  ): 
#     date_from = datetime.datetime(year_from, 3, 5)
#     date_to = datetime.datetime(year_to, 12, 30)
    
#     period_small = breakout_period #20
#     period_big = breakout_period #55
    
#     columns = ['Id', 'Ticker', 'Sector','Direct', 'Lot','Size/money_unit', 'Date/price_open', 'Date/price_close', 
#                'Criterion', 'Profit', 'ATR', 'N', 'IND']

#     data_actual_units = pd.DataFrame(columns=columns) # датафрейм с открытыми позициями
#     arr_actual_units = np.empty((0,15), int) # матрица с открытыми позициями
    
#     data_all_units =  pd.DataFrame(columns=columns) # датафрейм со всеми позициями
#     arr_all_units = np.empty((0,15), int) # матрица со всеми позициями (каждый массив из 15 значений)
    
#     stocks_dict_add = create_stocks_dict_add(stocks_list) # Сюда будут добавляться котировки

    
#     calendar = calendar_year(date_from, date_to) # Календарь
#     parallel_deals = [] # Колличество одновременно открытых сделок
    
#     test_list = []
#     test_list_2 = []
#     id_pos = 0 # Id сделки

#     ## Добавление индикатора в data
#     for i in stocks_dict_full.keys(): 
#         stocks_dict_full[i] = BB(stocks_dict_full[i])
    
    
#     for date in tqdm(calendar):
#         for ticker in stocks_list:
#             data_ticker = data_actual_units[data_actual_units.loc[:,'Ticker'] == ticker] # датафрейм тикера из общего датафрейма открытых позиций
#             stocks_dict_add[ticker] = pd.concat([stocks_dict_add[ticker], stocks_dict_full[ticker][stocks_dict_full[ticker].index == date]]) #Добавили строку в словарь котировок       
#             data = stocks_dict_add[ticker] # ДФ со значениями котировок (уже добавленных)
            
#             if len(data) > 0 and (date in list(data.index)): # Если есть котировки по акции
#                 N = N_mean(data, date) 
# #                 price_now = data.loc[date,'Open'] # Цена открытия (ищем дешевые акции)
#                 if len(data_ticker) > 0 : # Если есть открытые позиции
#                     id_list = list(data_ticker.loc[:,'Id'])
# #                     print(id_list)
#                     for i in id_list: # Для каждого юнита прогоняет все 3 условия (индексация по дате открытых позиций)
#                         data_actual_units, data_all_units = close_option(data_actual_units, data_all_units, stocks_dict_add, 
#                                                                          usd_stocks, ticker, date, min_max_period,
#                                                                          N, data_ticker, data, i) # Закрывает по минимум 10 дней
            
            
# #                         data_actual_units, data_all_units = close_profit(data_actual_units, data_all_units, stocks_dict_add, 
# #                                                                          usd_stocks, ticker, date, min_max_period,
# #                                                                          N, data_ticker, data, i) # Закрывает по минимум 10 дней
                      
# #                         data_actual_units, data_all_units = close_stop_loss(data_actual_units, data_all_units, stocks_dict_add, 
# #                                                                              usd_stocks, ticker, date, corr_stop_loss,
# #                                                                              N, data_ticker, data, i) # Закрывает по стоплоссу
                        
#                 b_long_small = breakout_long(period_small, stocks_dict_add, date, ticker, data)
#                 b_short_small = breakout_short(period_small, stocks_dict_add, date, ticker, data)

#      ###### Если сегодня есть прорыв канала, проверяем предыдущий прорыв на прибыльность. Условие LONG. Период 20 дней
#                 if (b_long_small == 1 and b_short_small != 1): # Условие, что прорыв только из при небольшом N
# #                     b_common_p = breakout_common_prev(period, stocks_dict_add, date, ticker) # Условие: предыдущтй прорыв был прибыльным
#                     check, count_ticker_corr = check_open_positions(data_actual_units, stocks_dict_add, usd_stocks, ticker, 'Long', date,
#                                                  count_direct, count_sector, count_all, delta_N_add, N, data_ticker, 
#                                                  count_ticker, n_range_min, n_range_max) 
                    
# ################################# ТУТ НАЧИНАЕТСЯ LONG  
#                     if (breakout_long(period_big, stocks_dict_add, date, ticker, data) == 1 # Был прорыв в большом периоде
# #                         and b_common_p == 1  # предыдущий прорыв был прибыльный  
#                         and check == 1 ):
#                         data_actual_units, id_pos = open_position_long(stocks_dict_add,
#                                                                       usd_stocks, 
#                                                                       data_actual_units, 
#                                                                       columns,
#                                                                       deposit,
#                                                                       date,
#                                                                       ticker, 
#                                                                       period_big, 
#                                                                       corr_unit_size, 
#                                                                       test_list,
#                                                                       id_pos, 
#                                                                       'Long',
#                                                                       N,
#                                                                       data,
#                                                                       mmd_period, 
#                                                                       count_ticker_corr)

#                         parallel_deals.append(data_actual_units[data_actual_units.loc[:, 'Ticker'] == ticker]['Ticker'].count())
                   
# #                     elif (b_common_p == 0  # предыдущий прорыв был убыточный  
# #                           and breakout_long(period_small, stocks_dict_add, date, ticker, data) == 1 
# #                           and check == 1):
# #                         data_actual_units, id_pos = open_position_long(stocks_dict_add,
# #                                                               usd_stocks,
# #                                                               data_actual_units, 
# #                                                               columns,
# #                                                               deposit,
# #                                                               date,
# #                                                               ticker,
# #                                                               period_small,
# #                                                               corr_unit_size,
# #                                                               test_list,
# #                                                               id_pos, 
# #                                                               'Long', 
# #                                                               N,
# #                                                               data,
# #                                                               mmd_period,
# #                                                               count_ticker_corr)

# #                         parallel_deals.append(data_actual_units[data_actual_units.loc[:, 'Ticker'] == ticker]['Ticker'].count())

# ################################# ТУТ НАЧИНАЕТСЯ SHORT      
#                 elif (b_short_small == 1 and b_long_small != 1):
# #                     b_common_p = breakout_common_prev(period, stocks_dict_add, date, ticker) # Условие: предыдущтй прорыв был прибыльным
#                     check, count_ticker_corr = check_open_positions(data_actual_units, stocks_dict_add, usd_stocks, ticker, 'Short', date,
#                                                  count_direct, count_sector, count_all, delta_N_add, N, data_ticker, 
#                                                  count_ticker, n_range_min, n_range_max) 
                   
                
#                     if (breakout_short(period_big, stocks_dict_add, date, ticker, data) == 1 # Был прорыв в большом периоде
# #                         and b_common_p == 1  # предыдущий прорыв был прибыльный  
#                         and check == 1):
#                         data_actual_units, id_pos = open_position_long(stocks_dict_add,
#                                                                        usd_stocks, 
#                                                                        data_actual_units, 
#                                                                        columns,
#                                                                        deposit,
#                                                                        date,
#                                                                        ticker, 
#                                                                        period_big, 
#                                                                        corr_unit_size, 
#                                                                        test_list,
#                                                                        id_pos, 
#                                                                        'Short', 
#                                                                        N,
#                                                                        data,
#                                                                        mmd_period,
#                                                                        count_ticker_corr)
#                         parallel_deals.append(data_actual_units[data_actual_units.loc[:, 'Ticker'] == ticker]['Ticker'].count())
                      
# #                     elif (b_common_p == 0 # предыдущий прорыв был убыточный  
# #                           and breakout_short(period_small, stocks_dict_add, date, ticker, data) == 1 
# #                           and check == 1):
# #                         data_actual_units, id_pos = open_position_long(stocks_dict_add,
# #                                                               usd_stocks,
# #                                                               data_actual_units, 
# #                                                               columns,
# #                                                               deposit,
# #                                                               date,
# #                                                               ticker,
# #                                                               period_small,
# #                                                               corr_unit_size,
# #                                                               test_list,
# #                                                               id_pos, 
# #                                                               'Short', 
# #                                                               N, 
# #                                                               data,
# #                                                               mmd_period, 
# #                                                               count_ticker_corr)
# # #                         data_actual_units, data_all_units = close_2N(data_actual_units, data_all_units, stocks_dict_add, 
# # #                                                                          usd_stocks, ticker, date, dev_max_long, 
# # #                                                                          N, data_ticker, data, id_pos) # Закрывает по отклонению 2N
                        
# #                         parallel_deals.append(data_actual_units[data_actual_units.loc[:, 'Ticker'] == ticker]['Ticker'].count())
                        
#                 data_actual_units, data_all_units = close_2N(data_actual_units, data_all_units, data_ticker, data, N, date,
#                                                              dev_max_long) 
# #             else:
# #                 print('date')
# #     display(data_all_units.loc[:,'Ticker'].unique())
# #     print('Тикеры открытых сделок:', list(set(test_list)))
#     display('Открытые сделки', data_actual_units)
#     unique_list = list(set(test_list))
#     print('-------------------------------------')
# #     display ('Все сделки', data_all_units.sort_index())
#     print('Уникальных инструментов:{}/{}'.format(len(data_all_units.loc[:,'Ticker'].unique()), len(unique_list)))
#     print ('Profit:', data_all_units.loc[:,'Profit'].sum())
#     print ('Максимальное колличество юнитов:', max(parallel_deals))
    
#     return data_all_units.sort_values(by='Id'), data_all_units.loc[:,'Profit'].sum() 